<a href="https://colab.research.google.com/github/gillesretiere/langdeck/blob/langdeckAB/langdeck_pipe_2024_v1_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
#!/usr/bin/env python
# -*- coding: utf8 -*-
import pandas as pd
pd.set_option("display.width",1000)
pd.options.mode.chained_assignment = None  # default='warn'

import numpy as np
import requests
import json
import uuid
from datetime import datetime
import datetime
import re
import string
import unicodedata
import ast

!pip3 install pyclean

# Gspread  API for Google sheets
!pip install gspread --upgrade
# fix here => https://stackoverflow.com/questions/71347973/modulenotfounderror-no-module-named-gspread-models
!pip uninstall -y gspread-dataframe
!pip install gspread-dataframe
!pip install gspread-formatting

#==== Google Drive
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/langdeck/
from google.colab import files
from google.colab import auth
auth.authenticate_user()

#====== Google Credentials ================
from oauth2client.client import GoogleCredentials
# fix here => https://github.com/burnash/gspread/issues/1014
from google.auth import default
creds, _ = default()

# python .env see -> https://saurabh-kumar.com/python-dotenv/
!pip install python-dotenv
from dotenv import load_dotenv
from dotenv import dotenv_values
load_dotenv()
dotenv_config = dotenv_values(".env")
app_path = dotenv_config["APP_PATH"]

#====== GSpread is a Python API for Google Sheets.================
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from gspread_formatting import *
gc = gspread.authorize(creds)

#====== Airtable ================
!pip install airtable-python-wrapper
from airtable import Airtable

#====== Deep Translate ================
!pip install -U deep-translator
!pip install --upgrade deepl

#====== Dictionaries ================
# larousse API
!pip install larousse-api-sunbro

#====== Lemmatizers ================
# https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

#====== Text To Speech
# about quotas : https://stackoverflow.com/questions/65980562/gtts-tts-gttserror-429-too-many-requests-from-tts-api-probable-cause-unknow
import os
!pip install gtts --upgrade
from gtts import gTTS

# CDN Cloudinary for upload
!pip3 install cloudinary
import cloudinary
import cloudinary.uploader
import cloudinary.api
from cloudinary.uploader import upload
from cloudinary.utils import cloudinary_url

!export PYTHONIOENCODING=utf8

%load_ext autoreload
%autoreload 2
%cd /content/drive/MyDrive/langdeck/
!pyclean .
import sys, os
sys.path.append('/content/drive/MyDrive/langdeck/py_modules')
import py_modules
py_modules.test()

# .env
from dotenv import load_dotenv
from dotenv import dotenv_values
%cd /content/drive/MyDrive/langdeck/
load_dotenv()
dotenv_config = dotenv_values(".env")

# Config CDN
cloudinary.config(
  cloud_name = dotenv_config["CLOUDINARY_CLOUD_NAME"],
  api_key = dotenv_config["CLOUDINARY_API_KEY"],
  api_secret = dotenv_config["CLOUDINARY_API_SECRET"],
  secure = True
)

#load_dotenv()
#dotenv_config = dotenv_values(".env")

print ("Environnement prêt.")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: gspread
    Found existing installation: gspread 6.0.2
    Uninstalling gspread-6.0.2:
      Successfully uninstalled gspread-6.0.2
Found existing installation: gspread-dataframe 3.3.1
Uninstalling gspread-dataframe-3.3.1:
  Successfully uninstalled gspread-dataframe-3.3.1
Mounted at /content/drive
/content/drive/MyDrive/langdeck
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00
  Cloning https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git to /tmp/pip-req-build-edij2p6q
  Running command git clone --filter=blob:none --quiet https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git /tmp/pip-req-build-edij2p6q
  Resolved https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git to commit bc0ebd0135a6cc78f48ddf184069b4c0b9c017d8
  Preparing metadata (setup.py) ... done
 

# Cnx Google Sheet DB
Ici on teste la cnx avec la base d'intégration qui est une table GSHEET

In [ ]:
db_lgdk_intgr = dotenv_config["DB_LGDK_INTGR"]
#--- integration database
sheet = db_lgdk_intgr
wb_intgr  = gc.open_by_url(sheet)
#--- Liste des codes ISO639-3
df_lang_alpha3 = py_modules.load_df_from_gsheet (wb_intgr, "ref_iso-639-3")
#--- Liste des langues (issues d'un Corpus)
df_lang_corpus = py_modules.load_df_from_gsheet (wb_intgr, "ref_lang_corpus")
# df_lang_corpus.head()

# Cnx Airtable DB
Ici on teste la cnx avec la base de DESIGN qui est une table Airtable
##### doc : https://airtable-python-wrapper.readthedocs.io/en/airtable-python-wrapper/index.html


In [ ]:
#---- Base Airtable 2023
# deprecation of API keys on February 1st, 2024
#api_key_airtable = dotenv_config["AIRTABLE_PIPE23_API_KEY"]
api_key_airtable = dotenv_config["AIRTABLE_PIPE23_PAT"]
#
headers = {"Authorization": "Bearer " + api_key_airtable,"Content-Type" : "application/json"}
base_id = dotenv_config["AIRTABLE_PIPE23_BASE_ID"]
table_name = dotenv_config["AIRTABLE_PIPE23_TBL_NAME"]
airtable = Airtable(base_id, table_name, api_key_airtable)
active_filter = "{Status}='Done'"

## Tableau phrases pour InDesign
[vers la section](#scrollTo=muMITT0Glmq8&line=1&uniqifier=1)

# Langues

In [ ]:
tbd_languages = Airtable(base_id, "languages", api_key_airtable)
lang_filter = "{lang_is_available}='TRUE'"
vw_lang_grid = tbd_languages.get_all(
    view='view_grid',
    sort=['lang_id','lang_is_available'],
    formula=lang_filter,
    )

In [ ]:
def get_audio_url_fr(rec_id, df, rec_tr_id, audio_url, trace):
  key = rec_id + "-fre"
  if trace:
    print (f"key is : {key}")
    print (f"rec_tr_id is : {rec_tr_id}")
    print (f"audio_url is : {audio_url}")
  value = np.nan
  try:
    value = df.loc[df[rec_tr_id]==key][audio_url].values.item()
    if trace:
      print(f"french audio url is : {value}")
  except Exception as e:
    print(f"error is : {str(e)}")
  return value

# Thèmes

## Intégration

In [ ]:
tbd_themes = Airtable(base_id, "themes", api_key_airtable)
vw_themes_grid = tbd_themes.get_all(
    view='view_grid',
    sort=['theme_name'],
    formula=active_filter,
    )

#=======================================================
# Transfo en dataframe
#=======================================================
df_themes = pd.DataFrame.from_records((r['fields'] for r in vw_themes_grid))
df_themes = df_themes[["theme_rec_id",
                 "theme_name",
                 "theme_desc",
                 "lessons",
                 "stories",
                 "theme_illustration",
                 ]]


#=======================================================
# Ajout des langues en colonnes
#=======================================================
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)
py_modules.save_df_to_gsheet (wb_intgr, "tbl_languages", df_languages)

# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_themes = pd.concat([df_themes, df_t.reindex(df_themes.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_themes,
        id_vars=["theme_rec_id",
                 "theme_name",
                 "theme_desc",
                 "lessons",
                 "stories",
                 "theme_illustration",
                 ],
        var_name="theme_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["theme_translation_id"] = df_w_temp[['theme_rec_id', 'theme_language']].apply(
    lambda x: "{}-{}".format(x.iloc[0],x.iloc[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)


#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_themes = py_modules.load_df_from_gsheet (wb_intgr, "tbl_themes")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_themes[["theme_translation_id",
                  "theme_translation",
                  "theme_translation_status",
                  "theme_audio",
                  "theme_audio_url",
                  "theme_audio_url_fr"]],
    on="theme_translation_id",
    how="left")

## Traduction

In [ ]:
df_w_tr["theme_translation"] = df_w_tr["theme_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["theme_translation_status"] = df_w_tr["theme_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["theme_translation","theme_translation_status"]] = df_w_tr[["theme_translation",
                                                                   "theme_name",
                                                                   "theme_language",
                                                                   "theme_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x.iloc[1],
        "fre",
        x.iloc[2],
        x.iloc[3],
        df_languages)
    if (pd.isna(x.iloc[0]) or pd.isna(x.iloc[3]) or x.iloc[3].lower()=="update")
    else (x.iloc[0],x.iloc[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["theme_audio"] = df_w_tr[["theme_translation_id","theme_language"]].apply(
    lambda x:"assets/audio/ai/"+x.iloc[1]+"/"+str(x.iloc[0])+".mp3",
    axis=1)

## Text To Speech

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["theme_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["theme_translation","theme_language","theme_audio","theme_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x.iloc[0],
    df_languages.loc[df_languages["lang_id"]==x.iloc[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x.iloc[1] + "/" + x.iloc[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x.iloc[1] + "/", x.iloc[2].split("/")[-1])==False
or x.iloc[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['theme_audio_url'] = df_tts['theme_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["theme_audio_url"] = df_tts[["theme_language","theme_audio","theme_audio_url","theme_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x.iloc[0],
    audio_path + x.iloc[0]+"/",
    x.iloc[1].split("/")[-1],
    x.iloc[3])
    if pd.isna(x.iloc[2]) or x.iloc[3].lower() == "update"
    else x.iloc[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["theme_audio_url_fr"] = df_tts["theme_rec_id"].map(
    lambda x:py_modules.theme_get_audio_url_fr(x, df_tts))

/content/drive/MyDrive/langdeck/assets/audio/ai/ams
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/rom
/content/drive/MyDrive/langdeck/assets/audio/ai/rus


In [ ]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_themes", df_tts)

# Lessons

## Intégration

In [ ]:
tbd_lessons = Airtable(base_id, "lessons", api_key_airtable)
vw_lessons_grid = tbd_lessons.get_all(
    view='view_grid',
    sort=['lesson'],
    formula=active_filter,
    )

#=======================================================
# Transfo en dataframe
#=======================================================
vk_col_lessons = ["lesson_rec_id",
      "lesson",
      "lesson_story",
      "lesson_pictorial",
      "lesson_quiz",
      "lesson_number",
      "lesson_illustration",
      "lesson_related_theme",
      "Status",
      ]
df_lessons = pd.DataFrame.from_records((r['fields'] for r in vw_lessons_grid))
df_lessons = df_lessons[vk_col_lessons]

In [ ]:
#=======================================================
# Ajout des langues en colonnes
#=======================================================
# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_lessons = pd.concat([df_lessons, df_t.reindex(df_lessons.index)], axis=1)

#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_lessons,
        id_vars=vk_col_lessons,
        var_name="lesson_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["lesson_translation_id"] = df_w_temp[['lesson_rec_id', 'lesson_language']].apply(
    lambda x: "{}-{}".format(x.iloc[0],x.iloc[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)

#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
vk_col_t_lessons = [
    "lesson_translation_id",
    "lesson_translation",
    "lesson_translation_status",
    "lesson_audio",
    "lesson_audio_url",
    "lesson_audio_url_fr"
    ]
df_tbl_lessons = py_modules.load_df_from_gsheet (wb_intgr, "tbl_lessons")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_lessons[vk_col_t_lessons],
    on="lesson_translation_id",
    how="left")


## Traduction

In [ ]:
df_w_tr["lesson_translation"] = df_w_tr["lesson_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["lesson_translation_status"] = df_w_tr["lesson_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["lesson_translation","lesson_translation_status"]] = df_w_tr[["lesson_translation",
                                                                   "lesson",
                                                                   "lesson_language",
                                                                   "lesson_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x.iloc[1],
        "fre",
        x.iloc[2],
        x.iloc[3],
        df_languages)
    if (pd.isna(x.iloc[0]) or pd.isna(x.iloc[3]) or x.iloc[3].lower()=="update")
    else (x.iloc[0],x.iloc[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["lesson_audio"] = df_w_tr[["lesson_translation_id","lesson_language"]].apply(
    lambda x:"assets/audio/ai/"+x.iloc[1]+"/"+str(x.iloc[0])+".mp3",
    axis=1)

## Text To Speech

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["lesson_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["lesson_translation","lesson_language","lesson_audio","lesson_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x.iloc[0],
    df_languages.loc[df_languages["lang_id"]==x.iloc[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x.iloc[1] + "/" + x.iloc[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x.iloc[1] + "/", x.iloc[2].split("/")[-1])==False
or x.iloc[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['lesson_audio_url'] = df_tts['lesson_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
df_tts["lesson_audio_url"] = df_tts[["lesson_language","lesson_audio","lesson_audio_url","lesson_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x.iloc[0],
    audio_path + x.iloc[0]+"/",
    x.iloc[1].split("/")[-1],
    x.iloc[3])
    if pd.isna(x.iloc[2]) or x.iloc[3].lower() == "update"
    else x.iloc[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["lesson_audio_url_fr"] = df_tts["lesson_rec_id"].map(
    lambda x:py_modules.table_get_audio_url_fr(x,
                                    df_tts,
                                    "lesson_translation_id",
                                    "lesson_audio_url",
                                    ))

/content/drive/MyDrive/langdeck/assets/audio/ai/ams
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/rom
/content/drive/MyDrive/langdeck/assets/audio/ai/rus


In [ ]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_lessons", df_tts)

# Stories

## Intégration

In [ ]:
tbd_stories = Airtable(base_id, "stories", api_key_airtable)
vw_stories_grid = tbd_stories.get_all(
    view='view_grid',
    sort=['story_name','story_level'],
    formula=active_filter,
    )
#=======================================================
# Transfo en dataframe
#=======================================================
df_stories = pd.DataFrame.from_records((r['fields'] for r in vw_stories_grid))
df_stories.rename(columns={'story_theme': 'story_theme_rec_id'}, inplace=True)
df_stories["story_theme"] = df_stories["story_theme_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_themes, x, "theme_name"),
    na_action="ignore"
)
vk_col_stories = ["story_rec_id",
                 "story_name",
                 "story_episode",
                 "story_level",
                 "story_theme",
                 "story_illustration",
                 "paragraphs",
                 "story_desc",
                 "story_desc_tr_status",
                 "story_synopsis",
                 "story_synopsis_tr_status",
                 "story_lesson_header",
                 "story_related_lesson",
                 ]
df_stories = df_stories[vk_col_stories]
#=======================================================
# Ajout des langues en colonnes
#=======================================================
# df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)

# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_stories = pd.concat([df_stories, df_t.reindex(df_stories.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_stories,
        id_vars=vk_col_stories,
        var_name="story_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["story_translation_id"] = df_w_temp[['story_rec_id', 'story_language']].apply(
    lambda x: "{}-{}".format(x.iloc[0],x.iloc[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)
#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_stories = py_modules.load_df_from_gsheet (wb_intgr, "tbl_stories")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_stories[["story_translation_id",
                  "story_translation",
                  "story_translation_status",
                  "story_audio",
                  "story_audio_url",
                  "story_audio_url_fr",
                  "story_desc_translation",
                  "story_synopsis_translation",
                  ]],
    on="story_translation_id",
    how="left")

## Traduction

In [ ]:
df_w_tr["story_translation"] = df_w_tr["story_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["story_translation_status"] = df_w_tr["story_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_w_tr[["story_translation","story_translation_status"]] = df_w_tr[["story_translation",
                                                                   "story_name",
                                                                   "story_language",
                                                                   "story_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x.iloc[1],
        "fre",
        x.iloc[2],
        x.iloc[3],
        df_languages)
    if (pd.isna(x.iloc[0]) or pd.isna(x.iloc[3]) or x.iloc[3].lower()=="update")
    else (x.iloc[0],x.iloc[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["story_audio"] = df_w_tr[["story_translation_id","story_language"]].apply(
    lambda x:"assets/audio/ai/"+x.iloc[1]+"/"+str(x.iloc[0])+".mp3",
    axis=1)

## Text To Speech

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)

vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["story_language"].isin(vk_tts)]
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["story_translation","story_language","story_audio","story_translation_status"]].apply(
    lambda x:py_modules.text2speech(
    x.iloc[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x.iloc[1] + "/" + x.iloc[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x.iloc[1] + "/", x.iloc[2].split("/")[-1])==False
or x.iloc[3].lower()=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['story_audio_url'] = df_tts['story_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["story_audio_url"] = df_tts[["story_language","story_audio","story_audio_url","story_translation_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x.iloc[0],
    audio_path + x.iloc[0]+"/",
    x.iloc[1].split("/")[-1],
    x.iloc[3])
    if pd.isna(x.iloc[2]) or x.iloc[3].lower() == "update"
    else x.iloc[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["story_audio_url_fr"] = df_tts["story_rec_id"].map(
    lambda x:py_modules.story_get_audio_url_fr(x, df_tts))

/content/drive/MyDrive/langdeck/assets/audio/ai/ams
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/rom
/content/drive/MyDrive/langdeck/assets/audio/ai/rus


### Translation of story description

In [ ]:
# translate if necessary # status == NaN OR START
df_tts["story_synopsis_translation"] = df_tts["story_synopsis_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_tts[["story_synopsis_translation","story_synopsis_tr_status"]] = df_tts[
    ["story_synopsis_translation",
     "story_synopsis",
     "story_language",
     "story_synopsis_tr_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x.iloc[1],
        "fre",
        x.iloc[2],
        x.iloc[3],
        df_languages)
    if ((pd.isna(x.iloc[0]) and not pd.isna(x.iloc[1])) or x.iloc[3]=='update')
    else (x.iloc[0],x.iloc[3]),
    axis=1,
    result_type="expand"
    )

# translate if necessary # status == NaN OR START
df_tts["story_desc_translation"] = df_tts["story_desc_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

df_tts[["story_desc_translation","story_desc_tr_status"]] = df_tts[
    ["story_desc_translation",
     "story_desc",
     "story_language",
     "story_desc_tr_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x.iloc[1],
        "fre",
        x.iloc[2],
        x.iloc[3],
        df_languages)
    if ((pd.isna(x.iloc[0]) and not pd.isna(x.iloc[1])) or x.iloc[3]=='update')
    else (x.iloc[0],x.iloc[3]),
    axis=1,
    result_type="expand"
    )

# if (pd.isna(x[0]) or pd.isna(x[3]) or x[3].lower()=="update")

In [ ]:
py_modules.save_df_to_gsheet (wb_intgr, "tbl_stories", df_tts)

# Words

In [ ]:
tbd_words = Airtable(base_id, "words", api_key_airtable)
vw_words_grid = tbd_words.get_all(
    view='view_grid',
    sort=['word','word_type'],
    formula=active_filter,
    )

## Intégration

In [ ]:
#=======================================================
# Transfo en dataframe
#=======================================================
df_words = pd.DataFrame.from_records((r['fields'] for r in vw_words_grid))
# on ne garde que les colonnes utiles
df_words = df_words[["word_rec_id",
                     "word",
                     "word_type",
                     "word_lemmas",
                     "word_notes",
                     "word_nutri_notes",
                     "word_notes_status",
                     "word_nutri_notes_status",
                     "word_status",
                     ]]
# Capitalisation
df_words["word"]=df_words["word"].map(lambda x:x[:1].upper() + x[1:])
# on convertit les rec_id en valeur litérale
df_words.rename(columns={'word_lemmas': 'word_lemmas_rec_id'}, inplace=True)
# on convertit les rec_id en valeur litérale
df_words["word_lemmas"] = df_words["word_lemmas_rec_id"].map(
    lambda x:py_modules.get_word_lemma_by_id(tbd_words, x),
    na_action="ignore")

#=======================================================
# Ajout des langues en colonnes
#=======================================================
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_words = pd.concat([df_words, df_t.reindex(df_words.index)], axis=1)

#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_w_temp = pd.melt(df_words,
        id_vars=["word_rec_id",
                 "word",
                 "word_type",
                 "word_lemmas",
                 "word_lemmas_rec_id",
                 "word_notes",
                 "word_nutri_notes",
                 "word_notes_status",
                 "word_nutri_notes_status",
                 "word_status",
                 ],
        var_name="word_language",
        value_name="translation")
# hash incl. language iso3
df_w_temp["word_translation_id"] = df_w_temp[['word_rec_id', 'word_language']].apply(
    lambda x: "{}-{}".format(x.iloc[0],x.iloc[1]),
    axis=1)
# left join to include translations
df_w_temp.drop(columns=["translation",], axis=1, inplace=True)
#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_words = py_modules.load_df_from_gsheet (wb_intgr, "tbl_words")
df_w_tr = pd.merge(
    df_w_temp,
    df_tbl_words[["word_translation_id",
                  "word_translation",
                  "word_translation_status",
                  "word_audio",
                  "word_audio_url",
                  "word_audio_url_fr",
                  "word_notes_translation",
                  "word_notes_translation_status",
                  "word_nutri_notes_translation",
                  "word_nutri_notes_translation_status",
                  ]],
    on="word_translation_id",
    how="left")


## Traduction

In [ ]:
df_w_tr["word_translation"] = df_w_tr["word_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["word_translation_status"] = df_w_tr["word_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# il faut forcer la maj car les lemmes ont peut-être changé => on recrée la colonne from scratch
df_w_tr["word_lemmas_tr_rec_id"]=df_w_tr[["word_lemmas_rec_id","word_language"]].apply(
    lambda x:py_modules.f_word_lemmas_tr_rec_id(x.iloc[0],x.iloc[1]) if type(x.iloc[0])==list else np.nan,
    axis=1)
# translate if necessary # status == NaN OR START
df_w_tr[["word_translation","word_translation_status"]] = df_w_tr[["word_translation",
                                                                   "word",
                                                                   "word_language",
                                                                   "word_translation_status",
                                                                   "word_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x.iloc[1],
        "fre",
        x.iloc[2],
        x.iloc[3],
        df_languages)
    if (pd.isna(x.iloc[0]) or pd.isna(x.iloc[3]) or x.iloc[3].lower()=="update" or x.iloc[4]=="update")
    else (x.iloc[0],x.iloc[3]),
    axis=1,
    result_type="expand"
    )

# on convertit les rec_id en valeur litérale
df_w_tr["word_lemmas_tr"] = df_w_tr["word_lemmas_tr_rec_id"].map(
    lambda x:py_modules.f_word_lemmas_tr(x, df_w_tr),
    na_action="ignore")

# emplacement du fichier audio correspondant au mot traduit
df_w_tr["word_audio"] = df_w_tr[["word_translation_id","word_language"]].apply(
    lambda x:"assets/audio/ai/"+x.iloc[1]+"/"+str(x.iloc[0])+".mp3",
    axis=1)



## Notes and Nutri notes

In [ ]:
df_w_tr["word_notes_translation"] = df_w_tr["word_notes_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["word_notes_translation_status"] = df_w_tr["word_notes_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

In [ ]:
# notes et nutri-notes
# translate if necessary # status == NaN OR START
df_w_tr[["word_notes_translation","word_notes_translation_status"]] = df_w_tr[
    ["word_notes_translation",
     "word_notes",
     "word_language",
     "word_notes_translation_status",
     "word_notes_status",
     ]].apply(
    lambda x:py_modules.word_translate_update_status(
        x.iloc[1],
        "fre",
        x.iloc[2],
        x.iloc[3],
        df_languages)
    if ((pd.isna(x.iloc[0]) and not pd.isna(x.iloc[1])) or x.iloc[3]=='update' or x.iloc[4]=='update')
    else (x.iloc[0],x.iloc[3]),
    axis=1,
    result_type="expand"
    )


Translated C’est une ... from fre to ar >>> إنه نظام غ...
Translated Document r... from fre to ar >>> وثيقة يكتب...
Translated Appareil e... from fre to ar >>> جهاز على ش...
Translated C’est une ... from fre to en >>> It is a va...
Translated Document r... from fre to en >>> Document w...
Translated Appareil e... from fre to en >>> A pen-shap...
Translated C’est une ... from fre to fr >>> C’est une ...
Translated Document r... from fre to fr >>> Document r...
Translated Appareil e... from fre to fr >>> Appareil e...
Translated C’est une ... from fre to ro >>> Este o die...
Translated Document r... from fre to ro >>> Document s...
Translated Appareil e... from fre to ro >>> Dispozitiv...
Translated C’est une ... from fre to ru >>> Это разноо...
Translated Document r... from fre to ru >>> Документ, ...
Translated Appareil e... from fre to ru >>> Устройство...


In [ ]:
df_w_tr["word_nutri_notes_translation"] = df_w_tr["word_nutri_notes_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_w_tr["word_nutri_notes_translation_status"] = df_w_tr["word_nutri_notes_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

In [ ]:
# notes et nutri-notes
# translate if necessary # status == NaN OR START
df_w_tr[["word_nutri_notes_translation","word_nutri_notes_translation_status"]] = df_w_tr[
    ["word_nutri_notes_translation",
     "word_nutri_notes",
     "word_language",
     "word_nutri_notes_translation_status"]].apply(
    lambda x:py_modules.word_translate_update_status(
        x.iloc[1],
        "fre",
        x.iloc[2],
        x.iloc[3],
        df_languages)
    if ((pd.isna(x.iloc[0]) and not pd.isna(x.iloc[1])) or x.iloc[3]=='update')
    else (x.iloc[0],x.iloc[3]),
    axis=1,
    result_type="expand"
    )


## Text To Speech

# /!\ Bug : le status est passé à Pending, donc on ne repasse jamais ici xar on teste status==update

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_w_tr.loc[df_w_tr["word_language"].isin(vk_tts)]

/content/drive/MyDrive/langdeck/assets/audio/ai/ams
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/rom
/content/drive/MyDrive/langdeck/assets/audio/ai/rus


In [ ]:

# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[["word_translation","word_language","word_audio","word_translation_status","word_status"]].apply(
    lambda x:py_modules.text2speech(
    x.iloc[0],
    df_languages.loc[df_languages["lang_id"]==x[1].lower()]["lang_alpha2"].values.item(),
    audio_path + x.iloc[1] + "/" + x.iloc[2].split("/")[-1],
    False)
if py_modules.is_audio(audio_path + x.iloc[1] + "/", x.iloc[2].split("/")[-1])==False
or x.iloc[3].lower()=="update"
or x.iloc[4]=="update"
else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['word_audio_url'] = df_tts['word_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["word_audio_url"] = df_tts[["word_language","word_audio","word_audio_url","word_translation_status","word_status"]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x.iloc[0],
    audio_path + x.iloc[0]+"/",
    x.iloc[1].split("/")[-1],
    x.iloc[3])
    if pd.isna(x.iloc[2]) or x.iloc[3].lower() == "update" or x.iloc[4]=="update"
    else x.iloc[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["word_audio_url_fr"] = df_tts["word_rec_id"].map(
    lambda x:py_modules.word_get_audio_url_fr(x, df_tts))

py_modules.save_df_to_gsheet (wb_intgr, "tbl_words", df_tts)

# Phrases

## Intégration

In [ ]:
tbd_phrases = Airtable(base_id, "phrases", api_key_airtable)
vw_phrases_grid = tbd_phrases.get_all(
    view='view_grid',
    sort=['phrase_related_story','phrase_paragraph','phrase_position'],
    formula=active_filter,
    )

#=======================================================
# Transfo en dataframe
#=======================================================
df_phrases = pd.DataFrame.from_records((r['fields'] for r in vw_phrases_grid))
df_phrases.rename(columns={'phrase_words': 'phrase_words_rec_id'}, inplace=True)
df_phrases["phrase_words"] = df_phrases["phrase_words_rec_id"].map(
    lambda x:py_modules.get_word_lemma_by_id(tbd_words, x),
    na_action="ignore")
# récupérer la saynète à partir de son id
df_phrases.rename(columns={'phrase_related_story': 'phrase_related_story_rec_id'}, inplace=True)
df_phrases["phrase_related_story"] = df_phrases["phrase_related_story_rec_id"].map(
    lambda x:py_modules.get_airtable_column_by_id(tbd_stories, x, "story_name"),
    na_action="ignore")

df_phrases["phrase_html"] = df_phrases[["phrase_rtf","phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html(x.iloc[0], x.iloc[1]),
    axis=1)

df_phrases["phrase_html"] = df_phrases[["phrase_html","phrase_words_rec_id"]].apply(
    lambda x:py_modules.phrase_words_span_rec_id(x.iloc[0], x.iloc[1])
    if type(x.iloc[1])==list
    else np.nan,
    axis=1)

df_phrases["phrase_html_rec_id"] = df_phrases[["phrase_rtf","phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html_kw(x.iloc[0], x.iloc[1]),
    axis=1)

df_phrases["phrase_html_rec_id"] = df_phrases[["phrase_html_rec_id","phrase_words_rec_id"]].apply(
    lambda x:py_modules.phrase_words_span_rec_id_kw(x.iloc[0], x.iloc[1])
    if type(x.iloc[1])==list
    else np.nan,
    axis=1)

df_phrases["phrase_html_kw"] = df_phrases[["phrase_rtf","phrase_rec_id"]].apply(
    lambda x:py_modules.phrase_words_markdown_to_html_kw(x.iloc[0], x.iloc[1]),
    axis=1)

In [ ]:
df_phrases["phrase_paragraph"] = df_phrases["phrase_paragraph"].str.zfill(2)
df_phrases["phrase_position"] = df_phrases["phrase_position"].str.zfill(2)

In [ ]:
vk_col_phrases = [
    "phrase_rec_id",
    "phrase",
    "phrase_words",
    "phrase_words_rec_id",
    "phrase_html",
    "phrase_html_rec_id",
    "phrase_html_kw",
    "phrase_related_story",
    "phrase_related_story_rec_id",
    "phrase_paragraph",
    "phrase_position",
    "phrase_status",
    "phrase_notes",
    "phrase_notes_2",
    "phrase_notes_status",
    "phrase_notes_2_status",
    "phrase_illustration",
    ]
df_phrases = df_phrases[vk_col_phrases]

#=======================================================
# Ajout des langues en colonnes
#=======================================================
# df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
# on transpose chaque trigramme de langue en une colonne supplémentaire (values n'a pas d'importance car on vide la table ensuite)
df_t = df_languages.pivot(columns='lang_id', values='lang_alpha3')
df_t = df_t[0:0]
df_phrases = pd.concat([df_phrases, df_t.reindex(df_phrases.index)], axis=1)
#=======================================================
#---- Transposition des colonnes de langues en lignes
#=======================================================
df_phr_temp = pd.melt(df_phrases,
        id_vars = vk_col_phrases,
        var_name = "phrase_language",
        value_name = "translation")
# hash incl. language iso3
df_phr_temp["phrase_translation_id"] = df_phr_temp[['phrase_rec_id', 'phrase_language']].apply(
    lambda x: "{}-{}".format(x.iloc[0],x.iloc[1]),
    axis=1)
# left join to include translations
df_phr_temp.drop(columns=["translation",], axis=1, inplace=True)

#=======================================================
#---- Merge avec les traductions existantes
#=======================================================
df_tbl_phrases = py_modules.load_df_from_gsheet (wb_intgr, "tbl_phrases")
df_phr_tr = pd.merge(
    df_phr_temp,
    df_tbl_phrases[[
        "phrase_translation_id",
        "phrase_translation",
        "phrase_translation_status",
        "phrase_audio",
        "phrase_audio_url",
        "phrase_audio_url_fr",
        "phrase_notes_translation",
        "phrase_notes_translation_status",
        "phrase_notes_2_translation",
        "phrase_notes_2_translation_status",
        ]],
    on="phrase_translation_id",
    how="left")

In [ ]:
df_phr_tr["phrase_notes_translation"] = df_phr_tr["phrase_notes_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_notes_translation_status"] = df_phr_tr["phrase_notes_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_notes_2_translation"] = df_phr_tr["phrase_notes_2_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_notes_2_translation_status"] = df_phr_tr["phrase_notes_2_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

## Traduction

### Traductions : mots à ne pas traduire
En attendant d'avoir les fonctionnalités de DeepL, on ignore les tags
```
def phrase_notes_markdown_to_html_kw (source):
  temp = re.sub(r'\n', '', source)
  temp = re.sub(r'\*+\`', f'<span translate="no">', temp)
  target = re.sub(r'\`\*+', '</span>', temp)
  return target
  
def phrase_notes_markdown_to_html_kw_2 (source):
  temp = re.sub(r'\n', '', source)
  temp = re.sub(r'\*+\`', f'<code>', temp)
  target = re.sub(r'\`\*+', '</code>', temp)
  return target
```



In [ ]:
# Phrase notes
b_phrase_notes = False
if b_phrase_notes:
  df_phr_tr["phrase_notes"] = df_phr_tr["phrase_notes"].map(lambda x:py_modules.phrase_notes_markdown_to_html_kw_2(x) if not pd.isna(x) else np.nan)
  df_phr_tr["phrase_notes_2"] = df_phr_tr["phrase_notes_2"].map(lambda x:py_modules.phrase_notes_markdown_to_html_kw_2(x) if not pd.isna(x) else np.nan)

  # translate if necessary # status == NaN OR START
  df_phr_tr[["phrase_notes_translation","phrase_notes_translation_status"]] = df_phr_tr[
      ["phrase_notes_translation",
      "phrase_notes",
      "phrase_language",
      "phrase_notes_translation_status",
      "phrase_notes_status",
      ]].apply(
      lambda x:py_modules.word_translate_update_status(
          x.iloc[1],
          "fre",
          x.iloc[2],
          x.iloc[3],
          df_languages)
      if ((pd.isna(x.iloc[0]) and not pd.isna(x.iloc[1])) or x.iloc[3]=='Start' or x.iloc[4]=='update')
      else (x.iloc[0],x.iloc[3]),
      axis=1,
      result_type="expand"
      )

  # translate if necessary # status == NaN OR START
  df_phr_tr[["phrase_notes_2_translation","phrase_notes_2_translation_status"]] = df_phr_tr[
      ["phrase_notes_2_translation",
      "phrase_notes_2",
      "phrase_language",
      "phrase_notes_2_translation_status",
      "phrase_notes_2_status",
      ]].apply(
      lambda x:py_modules.word_translate_update_status(
          x.iloc[1],
          "fre",
          x.iloc[2],
          x.iloc[3],
          df_languages)
      if ((pd.isna(x.iloc[0]) and not pd.isna(x.iloc[1])) or x.iloc[3]=='Start' or x.iloc[4]=='update')
      else (x.iloc[0],x.iloc[3]),
      axis=1,
      result_type="expand"
      )

In [ ]:
df_phr_tr["phrase_translation"] = df_phr_tr["phrase_translation"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
df_phr_tr["phrase_translation_status"] = df_phr_tr["phrase_translation_status"].map(
    lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)
# translate if necessary # status == NaN OR START
df_phr_tr[["phrase_translation","phrase_translation_status"]] = df_phr_tr[[
    "phrase_translation",
    "phrase",
    "phrase_language",
    "phrase_translation_status",
    "phrase_status",
    ]].apply(
    lambda x:py_modules.word_translate_update_status(
        x.iloc[1],
        "fre",
        x.iloc[2],
        x.iloc[3],
        df_languages)
    if (pd.isna(x.iloc[0]) or pd.isna(x.iloc[3]) or x.iloc[3]=='update' or x.iloc[4]=="update")
    else (x.iloc[0],x.iloc[3]),
    axis=1,
    result_type="expand"
    )

# emplacement du fichier audio correspondant au mot traduit
df_phr_tr["phrase_audio"] = df_phr_tr[["phrase_translation_id","phrase_language"]].apply(
    lambda x:"assets/audio/ai/"+x.iloc[1]+"/"+str(x.iloc[0])+".mp3",
    axis=1)

## Text To Speech

In [ ]:
# check language directories
audio_path = "/content/drive/MyDrive/langdeck/assets/audio/ai/"
df_languages = pd.DataFrame.from_records((r['fields'] for r in vw_lang_grid))
df_languages["lang_id"].map(lambda x:py_modules.create_gdrive_folder(audio_path, x))
# filtre sur les langues disponibles pour TTS (subset)
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x.upper()=="TRUE" else False)
vk_tts = df_languages.loc[df_languages["lang_is_tts"]]["lang_id"].to_list()
df_tts = df_phr_tr.loc[df_phr_tr["phrase_language"].isin(vk_tts)]
df_tts["phrase_translation_status"] = df_tts[["phrase_translation_status","phrase_status"]].apply(
    lambda x:
    x.iloc[1] if x.iloc[1].lower() == "update" else x.iloc[0],
    axis=1)
# Appel TTS sur le subset (ex) : text2speech("Diet", "et", audio_path +"est" + "/" + "recIFmq8UmSGnpw0v-est.mp3", False)
df_tts[[
    "phrase_translation",
    "phrase_language",
    "phrase_audio",
    "phrase_translation_status",
    ]].apply(
    lambda x:py_modules.text2speech(
      x.iloc[0],
      df_languages.loc[df_languages["lang_id"]==x.iloc[1].lower()]["lang_alpha2"].values.item(),
      audio_path + x.iloc[1] + "/" + x.iloc[2].split("/")[-1],
      False
    )
    if py_modules.is_audio(audio_path + x.iloc[1] + "/", x.iloc[2].split("/")[-1])==False
      or x.iloc[3].lower()=="update"
    else None,
    axis=1)

# replace spaces or empty string by Nan
df_tts['phrase_audio_url'] = df_tts['phrase_audio_url'].map(
    lambda x: np.nan
    if isinstance(x, str) and (x.isspace() or not x)
    else x)

#=======================================================
# UPLOAD TO CLOUDINARY
#=======================================================

# upload only if audio_public_url equals NaN (if url exists, file has yet been uploaded)
#df_tts.loc[pd.isna(df_tts["audio_public_url"])]
df_tts["phrase_audio_url"] = df_tts[[
    "phrase_language",
    "phrase_audio",
    "phrase_audio_url",
    "phrase_translation_status",
    ]].apply(
    lambda x: py_modules.upload_to_cdn(
    cloudinary,
    x.iloc[0],
    audio_path + x.iloc[0]+"/",
    x.iloc[1].split("/")[-1],
    x.iloc[3])
    if pd.isna(x.iloc[2])
      or x.iloc[3].lower() == "update"
    else x.iloc[2],
    axis=1)

# cette fonction donne pour le mot l'url de la version en français
df_tts["phrase_audio_url_fr"] = df_tts["phrase_rec_id"].map(
    lambda x:py_modules.phrase_get_audio_url_fr(x, df_tts))

/content/drive/MyDrive/langdeck/assets/audio/ai/ams
/content/drive/MyDrive/langdeck/assets/audio/ai/eng
/content/drive/MyDrive/langdeck/assets/audio/ai/fre
/content/drive/MyDrive/langdeck/assets/audio/ai/rom
/content/drive/MyDrive/langdeck/assets/audio/ai/rus


In [ ]:
# récupère le mot traduit depuis son identifiant
def set_phrase_words_tr(rec_id, dfw):
  vk = []
  print (rec_id)
  for elem in rec_id:
    try:
      new_elem = dfw.loc[dfw["word_translation_id"]==elem]["word_translation"].values.item()
    except Exception as e:
      print (str(e) + " " + elem)
      raise
    vk.append(new_elem)
    print (new_elem)
  return vk

In [ ]:
# récupère le mot traduit depuis son identifiant
def set_phrase_words_tr(rec_id, dfw):
  vk = []
  for elem in rec_id:
    print (elem)
    try:
      vk.append(dfw.loc[dfw["word_translation_id"]==elem]["word_translation"].values.item())
    except Exception as e:
      pass
  return vk

In [ ]:
#=======================================================
# TRADUCTION DES MOTS ASSOCIES
#=======================================================
df_tbl_stories = py_modules.load_df_from_gsheet (wb_intgr, "tbl_stories")
df_tbl_words = py_modules.load_df_from_gsheet (wb_intgr, "tbl_words")
# # construit un id concaténé avec le code langue pour chaque element du vecteur
df_tts["phrase_words_tr_rec_id"] = df_tts[["phrase_words_rec_id","phrase_language"]].apply(
    lambda x:py_modules.set_phrase_words_tr_rec_id(
        x.iloc[0],
        x.iloc[1])
    if type(x.iloc[0])==list
    else np.nan,
    axis=1)
# récupère le mot traduit depuis son identifiant
df_tts["phrase_words_tr"] = df_tts["phrase_words_tr_rec_id"].map(
    lambda x:py_modules.set_phrase_words_tr(x, df_tbl_words),
    na_action='ignore')
#=======================================================
# TRADUCTION DES META DONNEES (STORY, THEME, ETC.)
#=======================================================
df_tts["phrase_related_story_tr_rec_id"] = df_tts[["phrase_related_story_rec_id","phrase_language"]].apply(
    lambda x:py_modules.set_phrase_words_tr_rec_id(
        x.iloc[0],
        x.iloc[1])
    if type(x.iloc[0])==list
    else np.nan,
    axis=1)

# récupère le mot traduit depuis son identifiant
df_tts["phrase_related_story_tr"] = df_tts["phrase_related_story_tr_rec_id"].map(
    lambda x:py_modules.get_story_tr(x, df_tbl_stories),
    na_action='ignore')

In [ ]:
# on rebascule les états si il y a eu update
df_tts["phrase_translation_status"] = df_tts[["phrase_translation_status","phrase_status"]].apply(
    lambda x:
    "Pending" if x.iloc[1].lower() == "update" else x.iloc[0],
    axis=1)

py_modules.save_df_to_gsheet (wb_intgr, "tbl_phrases", df_tts)

# Transformation format JSON
Chargement et transformation des tables en dictionnaires JSON

## Langues

In [ ]:
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages")
# merge pour récupérer des infos de référentiels
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","country_ref","flag_icon"]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid",], axis=1, inplace=True)
df_languages.rename(
    columns={"country_ref":"lang_country_ref",
             "flag_icon":"lang_flag_icon"},
    inplace=True)
# merge pour récupérer des infos de référentiels
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages_countries")
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","language_countries",]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid","lang_countries_vk",], axis=1, inplace=True)
df_languages.rename(
    columns={"language_countries":"lang_countries_vk",},
    inplace=True)
# Liste des colonnes pour le dict
vk_language_cols = [
    "lang_id",
    "lang_alpha3",
    "lang_alpha2_google",
    "lang_alpha2",
    "lang_wals",
    "lang_name_native",
    "lang_name_en",
    "lang_name_fr",
    "lang_is_available",
    "lang_is_tts",
    "lang_countries_vk",
    "lang_country_ref",
    "lang_flag_icon",
]
vk_lang_vals = df_languages.groupby(["lang_id"])[vk_language_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Chats, Steps & Options

In [ ]:
b_chat = False

In [ ]:
if b_chat:
  df_chats = py_modules.load_df_from_gsheet(wb_intgr, "tbl_xy_chats")
  df_chats.sort_values (
      by=["conversation",
          "conversation_language",
          ],
      inplace=True)

  df_chats['num_conversation'] = df_chats['num_conversation'].apply(lambda x: x.zfill(2))
  df_chats.sort_values(by=['conversation_language','num_conversation',], inplace=True)

  vk_chats_cols = [
      "conversation_rec_id",
      "conversation",
      "num_conversation",
      "conversation_illustration",
      "conversation_language",
      "conversation_translation_id",
      "conversation_translation",
      "conversation_audio",
      "conversation_audio_url",
      "conversation_audio_url_fr",
      ]

  vk_chats_vals = df_chats.groupby(["num_conversation"])[vk_chats_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()

In [ ]:
if b_chat:
  df_steps = py_modules.load_df_from_gsheet(wb_intgr, "tbl_xy_steps")
  df_steps.sort_values (
      by=["step",
          "step_language",
          ],
      inplace=True)

  df_steps["end_step"] = df_steps["end_step"].map(
      lambda x: np.nan if isinstance(x, str) and (x.isspace() or not x) else x)

  df_steps['num_step'] = df_steps['num_step'].apply(lambda x: x.zfill(2))

  df_steps['from_option'] = df_steps['from_option'].map(lambda x:ast.literal_eval(x) if x else [])
  df_steps['from_option'] = df_steps['from_option'].map(lambda x:''.join (x))
  df_steps['option'] = df_steps['option'].map(lambda x:ast.literal_eval(x) if x else [])
  df_steps['option'] = df_steps['option'].map(lambda x:''.join (x))
  df_steps['conversation'] = df_steps['conversations'].map(lambda x:ast.literal_eval(x) if x else [])
  df_steps['conversation'] = df_steps['conversation'].map(lambda x:''.join (x))
  df_steps['conversation_rec_id'] = df_steps['conversation_rec_id'].map(lambda x:ast.literal_eval(x) if x else [])
  df_steps['conversation_rec_id'] = df_steps['conversation_rec_id'].map(lambda x:''.join (x))

  df_steps.sort_values(by=['step_language','num_step',], inplace=True)

  vk_steps_cols = [
      "step_rec_id",
      "step",
      "num_step",
      "step_language",
      "step_translation_id",
      "step_translation",
      "end_step",
      "from_option",
      "option",
      "conversation",
      "conversation_rec_id",
      "step_audio",
      "step_audio_url",
      "step_audio_url_fr",
      ]

  vk_steps_vals = df_steps.groupby(["num_step"])[vk_steps_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()

In [ ]:
if b_chat:
  df_options = py_modules.load_df_from_gsheet(wb_intgr, "tbl_xy_options")

  df_options['num_from_step'] = df_options['num_from_step'].map(lambda x:ast.literal_eval(x) if x else [])
  df_options['num_from_step'] = df_options['num_from_step'].map(lambda x:''.join (x))
  df_options['from_step'] = df_options['from_step'].map(lambda x:ast.literal_eval(x) if x else [])
  df_options['from_step'] = df_options['from_step'].map(lambda x:''.join (x))
  df_options['goto_step'] = df_options['goto_step'].map(lambda x:ast.literal_eval(x) if x else [])
  df_options['goto_step'] = df_options['goto_step'].map(lambda x:''.join (x))
  df_options['num_goto_step'] = df_options['num_goto_step'].map(lambda x:ast.literal_eval(x) if x else [])
  df_options['num_goto_step'] = df_options['num_goto_step'].map(lambda x:''.join (x))
  df_options['goto_step_rec_id'] = df_options['goto_step_rec_id'].map(lambda x:ast.literal_eval(x) if x else [])
  df_options['goto_step_rec_id'] = df_options['goto_step_rec_id'].map(lambda x:''.join (x))

  df_options.sort_values (
      by=[
          "option_language",
          "num_from_step",
          ],
      inplace=True)

  df_options['num_from_step'] = df_options['num_from_step'].apply(lambda x: x.zfill(2))
  df_options['num_goto_step'] = df_options['num_goto_step'].apply(lambda x: x.zfill(2))

  vk_options_cols = [
      "option_rec_id",
      "option",
      "num_from_step",
      "from_step",
      "num_goto_step",
      "goto_step",
      "goto_step_rec_id",
      "option_language",
      "option_translation_id",
      "option_translation",
      "option_audio",
      "option_audio_url",
      "option_audio_url_fr",
      ]

  vk_options_vals = df_options.groupby([
          "option_language",
          "num_from_step",
          ])[vk_options_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()

In [ ]:
def df_to_dict__options (df, vk_cols, filter_language, col_language, rec_id, sort_col, trace, ):
  if trace:
    print (f'>> df_to_dict__options : {filter_language} {col_language} {rec_id} {sort_col}')
  vk_dict_returned = []
  df_filtered = df.loc[(df[col_language]==filter_language) & (df['from_step']==rec_id)]
  vk_vals = df_filtered.groupby([sort_col])[vk_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  if trace:
    print (len(vk_vals))
  for vk in vk_vals:
    if trace:
      print (f"parent rec_id is : {rec_id}")
    ## colonnes suppl.
    vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
    for vk_kv in vk_dict:
      d_key = {}
      if trace:
        print (vk_kv)
      for vkcol in vk_cols:
        d_key[vkcol] = vk_kv[vkcol]

      vk_dict_returned.append(d_key)

  return vk_dict_returned

In [ ]:
# df_steps.loc[(df_steps['step_language']=='dut') & (df_steps['conversation_rec_id']=='recvXw5xaOB1ZzFG3')]

In [ ]:
def df_to_dict__steps (df, vk_cols, filter_language, col_language, rec_id, sort_col, trace,):
  if trace:
    print (f'>> df_to_dict__steps : {filter_language} {col_language} {rec_id}')
  vk_dict_returned = []
  #df_filtered = df.loc[df[col_language]==filter_language]
  df_filtered = df.loc[(df[col_language]==filter_language) & (df['conversation_rec_id']==rec_id)]
  if len(df_filtered) == 0:
    return vk_dict_returned
  vk_vals = df_filtered.groupby([sort_col])[vk_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  for vk in vk_vals:
    vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
    d_key = {}

    ## colonnes suppl.
    for vkcol in vk_cols:
      if trace:
        print (vkcol)
      d_key[vkcol] = list(dict.fromkeys(map(
        lambda x:x[vkcol],
        sorted(vk_dict, key=lambda x: x[sort_col]))))[0]

    # on ajoute les options
    d_key["options"] = []
    if d_key["end_step"] is False or pd.isna(d_key["end_step"]):
      d_key["end_step"]='False'
      d_key["options"] = df_to_dict__options (
          df_options,
          vk_options_cols,
          d_key['step_language'],
          "option_language",
          d_key["step_rec_id"],
          "num_from_step",
          True,
          )

    vk_dict_returned.append(d_key)

  return vk_dict_returned

In [ ]:
def df_to_dict__chats (df, vk_cols, filter_language, col_language, rec_id, sort_col, trace,):
  if trace:
    print (f'>> df_to_dict__chats : {filter_language}')
  vk_dict_returned = []
  df_filtered = df.loc[df[col_language]==filter_language]
  vk_vals = df_filtered.groupby([sort_col])[vk_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  for vk in vk_vals:
    vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
    d_key = {}

    ## colonnes suppl.
    for vkcol in vk_cols:
      if trace:
        print (vkcol)
      d_key[vkcol] = list(dict.fromkeys(map(
        lambda x:x[vkcol],
        sorted(vk_dict, key=lambda x: x[sort_col]))))[0]

    # on ajoute les steps
    d_key['steps'] = df_to_dict__steps (
        df_steps,
        vk_steps_cols,
        d_lang['language'],
        "step_language",
        d_key["conversation_rec_id"],
        "num_step",
        True,
        )

    vk_dict_returned.append(d_key)

  return vk_dict_returned

In [ ]:
if b_chat:
  vk_langdeck_AB = []
  ### languages
  for vk_l in vk_lang_vals:
    vk_l_dict = list(map(lambda x: dict(zip(vk_language_cols,x)), vk_l))
    d_lang = {}
    d_lang["language"] = list(dict.fromkeys(map(
        lambda x:x["lang_id"],
        sorted(vk_l_dict, key=lambda x: x["lang_id"]))))[0]
    ## colonnes suppl.
    for vkcol in vk_language_cols:
      d_lang[vkcol] = list(dict.fromkeys(map(
        lambda x:x[vkcol],
        sorted(vk_l_dict, key=lambda x: x["lang_id"]))))[0]
    ## theme is the entry point
    d_lang['chats'] = df_to_dict__chats (
        df_chats,
        vk_chats_cols,
        d_lang['language'],
        "conversation_language",
        "conversation_rec_id",
        "num_conversation",
        True,
        )
    vk_langdeck_AB.append (d_lang)

In [ ]:
if b_chat:
  with open('langdeck_ab.txt', 'w') as f:
    json.dump(vk_langdeck_AB, f, ensure_ascii=False)

## Themes

In [ ]:
df_themes = py_modules.load_df_from_gsheet(wb_intgr, "tbl_themes")
df_themes.sort_values (
    by=["theme_name",
        "theme_language",
        ],
    inplace=True)

vk_themes_cols = [
    "theme_rec_id",
    "theme_name",
    "theme_language",
    "theme_translation",
    "theme_audio",
    "theme_audio_url",
    "theme_audio_url_fr",
    "theme_illustration",
    ]

vk_themes_vals = df_themes.groupby(["theme_rec_id"])[vk_themes_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Lessons

In [ ]:
df_lessons = py_modules.load_df_from_gsheet(wb_intgr, "tbl_lessons")
df_lessons.sort_values (
    by=["lesson_number",
        "lesson_language",
        ],
    inplace=True)

df_lessons['lesson_related_theme'] = df_lessons['lesson_related_theme'].map(lambda x:ast.literal_eval(x) if x else [])
df_lessons['lesson_related_theme'] = df_lessons['lesson_related_theme'].map(lambda x:''.join (x))

vk_lessons_cols = [
    "lesson_rec_id",
    "lesson",
    "lesson_language",
    "lesson_story",
    "lesson_pictorial",
    "lesson_quiz",
    "lesson_number",
    "lesson_translation",
    "lesson_audio",
    "lesson_audio_url",
    "lesson_audio_url_fr",
    "lesson_illustration",
    "lesson_related_theme",
    ]

vk_lessons_vals = df_lessons.groupby(["lesson_rec_id"])[vk_lessons_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Stories

In [ ]:
df_stories = py_modules.load_df_from_gsheet(wb_intgr, "tbl_stories")
df_stories.sort_values (
    by=["story_language",
        "story_level",
        "story_name",
        "story_episode",],
    inplace=True)

df_stories['story_related_lesson'] = df_stories['story_related_lesson'].map(lambda x:ast.literal_eval(x) if x else [])
df_stories['story_related_lesson'] = df_stories['story_related_lesson'].map(lambda x:''.join (x))

vk_stories_cols = [
    "story_rec_id",
    "story_translation_id",
    "story_name",
    "story_language",
    "story_translation",
    "story_level",
    "story_episode",
    "story_audio_url",
    "story_audio_url_fr",
    "story_illustration",
    "story_desc",
    "story_desc_translation",
    "story_synopsis",
    "story_synopsis_translation",
    "story_lesson_header",
    "story_related_lesson",
    ]

vk_stories_vals = df_stories.groupby(["story_rec_id"])[vk_stories_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Phrases

In [ ]:
df_phrases = py_modules.load_df_from_gsheet(wb_intgr, "tbl_phrases") # load source sheet
df_phrases["phrase_related_story"] = df_phrases["phrase_related_story"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_related_story_rec_id"] = df_phrases["phrase_related_story_rec_id"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_related_story_tr"] = df_phrases["phrase_related_story_tr"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_related_story_tr_rec_id"] = df_phrases["phrase_related_story_tr_rec_id"].map(lambda x:ast.literal_eval(x)[0])
df_phrases["phrase_paragraph"] = df_phrases["phrase_paragraph"].str.zfill(2)
df_phrases["phrase_position"] = df_phrases["phrase_position"].str.zfill(2)
df_themes = py_modules.load_df_from_gsheet(wb_intgr, "tbl_themes") # load source sheet
df_phrases.sort_values(by=['phrase_language','phrase_related_story','phrase_paragraph','phrase_position'], inplace=True)

# merge phrases + story => vue à plat avec les données étendues
df_phr_ext = pd.merge(df_phrases, df_stories[vk_stories_cols],
         left_on="phrase_related_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")

# vector of columns
vk_phrases_cols = [
    "phrase_language",
    "phrase_related_story",
    "phrase_related_story_rec_id",
    "phrase_paragraph",
    "phrase_position",
    "phrase_rec_id",
    "phrase",
    "phrase_words",
    "phrase_words_rec_id",
    "phrase_words_tr",
    "phrase_words_tr_rec_id",
    "phrase_html",
    "phrase_html_rec_id",
    "phrase_html_kw",
    "phrase_translation",
    "phrase_audio",
    "phrase_audio_url",
    "phrase_audio_url_fr",
    "phrase_related_story_tr",
    "phrase_notes",
    "phrase_notes_2",
    "phrase_notes_translation",
    "phrase_notes_2_translation",
    "phrase_illustration",
    ] + vk_stories_cols

    # fields (columns)

# vector of values grouped by language
vk_phrases_vals = df_phr_ext.groupby(["phrase_language","phrase_related_story"])[vk_phrases_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()


## Words

In [ ]:
df_words = py_modules.load_df_from_gsheet(wb_intgr, "tbl_words")
df_words["order"] = df_words["word_type"].map(lambda x:1 if x=="Lemma" else 2 if x=="Expression" else 3)
df_words.sort_values(by=['word_language','word','order',], inplace=True)
vk_word_cols = [
    "word_language",
    "word",
    "word_rec_id",
    "order",
    "word_type",
    "word_translation_id",
    "word_translation",
    "word_lemmas",
    "word_lemmas_rec_id",
    "word_audio",
    "word_audio_url",
    "word_audio_url_fr",
    "word_lemmas_tr_rec_id",
    "word_lemmas_tr",
    "word_notes",
    "word_nutri_notes",
    "word_notes_translation",
    "word_nutri_notes_translation",
    ] # fields (columns)

vk_word_vals = df_words.groupby(["word_language"])[vk_word_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## TPs

In [ ]:
df_tps = py_modules.load_df_from_gsheet(wb_intgr, "tbl_tps")
df_tps.sort_values (
    by=["tp_name",
        "tp_language",
        ],
    inplace=True)

vk_tps_cols = [
    "tp_rec_id",
    "tp_name",
    "tp_language",
    "tp_rel_story",
    "tp_rel_story_rec_id",
    "tp_illustration",
    "tp_translation",
    "tp_translation_id",
    "tp_audio",
    "tp_audio_url",
    "tp_audio_url_fr",
    "tp_lesson_header",
    "tp_summary",
    "tp_summary_translation",
    ]

#vk_tps_vals = df_tps.groupby(["tp_rec_id"])[vk_tps_cols].apply(
#      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_tps["tp_rel_story_rec_id"] = df_tps['tp_rel_story_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_tps["tp_rel_story"] = df_tps['tp_rel_story'].map(lambda x: "".join(map(str, ast.literal_eval(x))))

df_tps["tp_rel_story_tr_rec_id"] = df_tps[['tp_rel_story_rec_id', 'tp_language']].apply(
    lambda x: "{}-{}".format(x.iloc[0],x.iloc[1]),
    axis=1)

# merge phrases + story => vue à plat avec les données étendues
df_tps_ext = pd.merge(df_tps, df_stories[vk_stories_cols],
         left_on="tp_rel_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")

# vector of values grouped by language
vk_tps_vals = df_tps_ext.groupby(["tp_language","tp_name"])[vk_tps_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Phrases des TPs

In [ ]:
df_tp_phrases = py_modules.load_df_from_gsheet(wb_intgr, "tbl_tp_phrases")
df_tp_phrases.sort_values (
    by=["tp_phrase_language",
        "tp_phrase_order",
        ],
    inplace=True)

vk_tp_p_cols = [
    "tp_phrase_rec_id",
    "tp_phrase",
    "tp_phrase_html",
    "tp_phrase_html_rec_id",
    "tp_phrase_html_kw",
    "tp_phrase_rel_tp",
    "tp_phrase_rel_tp_rec_id",
    "tp_phrase_order",
    "tp_phrase_language",
    "tp_phrase_translation_id",
    "tp_phrase_translation",
    "tp_phrase_audio",
    "tp_phrase_audio_url",
    "tp_phrase_audio_url_fr",
    "tp_phrase_words_rec_id",
    "tp_phrase_words",
    "tp_phrase_words_tr_rec_id",
    "tp_phrase_words_tr",
    "phrase_related_tp_tr",
    ]

vk_tp_p_vals = df_tp_phrases.groupby(["tp_phrase_rec_id"])[vk_tp_p_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_tp_phrases["tp_phrase_rel_tp_rec_id"] = df_tp_phrases['tp_phrase_rel_tp_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_tp_phrases["tp_phrase_rel_tp"] = df_tp_phrases['tp_phrase_rel_tp'].map(lambda x: "".join(map(str, ast.literal_eval(x))))

# merge phrases + story => vue à plat avec les données étendues
df_tp_p_ext = pd.merge(df_tp_phrases, df_tps,
         left_on="tp_phrase_rel_tp_rec_id",
         right_on="tp_translation_id",
         how="left")

vk_col_tp_p_ext = vk_tp_p_cols + vk_tps_cols + [
  "tp_phrase_words_tr_rec_id",
  "tp_phrase_words_tr",
  "tp_rel_story_tr_rec_id",
  ]

# vector of values grouped by language
vk_tp_p_vals = df_tp_p_ext.groupby(["tp_phrase_language","tp_phrase_rel_tp"])[vk_col_tp_p_ext].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Quiz

In [ ]:
df_quiz = py_modules.load_df_from_gsheet(wb_intgr, "tbl_quiz")
df_quiz.sort_values (
    by=["quiz_name",
        "quiz_language",
        ],
    inplace=True)

vk_quiz_cols = [
    "quiz_rec_id",
    "quiz_name",
    "quiz_language",
    "quiz_rel_tp",
    "quiz_rel_tp_rec_id",
    "quiz_rel_story",
    "quiz_rel_story_rec_id",
    "quiz_illustration",
    "quiz_translation",
    "quiz_translation_id",
    "quiz_audio",
    "quiz_audio_url",
    "quiz_audio_url_fr",
    "quiz_summary_translation",
    "quiz_form",
    "quiz_form_translation",
    "quiz_lesson_header",
    ] + vk_stories_cols

df_quiz["quiz_rel_tp_rec_id"] = df_quiz['quiz_rel_tp_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_tp"] = df_quiz['quiz_rel_tp'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_story_rec_id"] = df_quiz['quiz_rel_story_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_story"] = df_quiz['quiz_rel_story'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_form"] = df_quiz["quiz_form"].map (lambda x:ast.literal_eval(x))
df_quiz["quiz_form_translation"] = df_quiz["quiz_form_translation"].map (lambda x:ast.literal_eval(x))

df_quiz["quiz_rel_tp_tr_rec_id"] = df_quiz[['quiz_rel_tp_rec_id', 'quiz_language']].apply(
    lambda x: "{}-{}".format(x.iloc[0],x.iloc[1]),
    axis=1)
df_quiz["quiz_rel_story_tr_rec_id"] = df_quiz[['quiz_rel_story_rec_id', 'quiz_language']].apply(
    lambda x: "{}-{}".format(x.iloc[0],x.iloc[1]),
    axis=1)

df_quiz_ext = pd.merge(df_quiz, df_stories[vk_stories_cols],
         left_on="quiz_rel_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")

# vector of values grouped by language
vk_quiz_vals = df_quiz_ext.groupby(["quiz_language"])[vk_quiz_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

<ipython-input-62-b0e5de9ee04b>:36: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: "{}-{}".format(x[0],x[1]),
<ipython-input-62-b0e5de9ee04b>:39: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  lambda x: "{}-{}".format(x[0],x[1]),


## Fonctions de remplissage au format JSON

In [ ]:
def add_dict_of_words (df_words,vk_word_cols, filter_language, vk_filter_words):
  vk_dict_story_phrase_words = []    # on crée un vecteur (liste) pour recevoir les mots de la phrase
  # on filtre sur langue ET identifiants des mots
  # print (len(vk_filter_words))
  if vk_filter_words :
    df_story_phrase_words_filtered = df_words.loc[
        (df_words['word_language']==filter_language) &
        (df_words['word_rec_id'].isin(vk_filter_words))
        ]
      # on créé un vecteur de tuples sur cette liste filtrée
    vk_story_phrase_words = df_story_phrase_words_filtered.groupby(["word_language",])[vk_word_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()
    # on itère le vecteur de mots => transforme le vecteur de tuples en liste de dicts
    for vk_w in vk_story_phrase_words:
      # print (f'phrase={vk_w}')
      vk_w_dict = list(map(lambda x: dict(zip(vk_word_cols,x)), vk_w))
      # on affecte chaque k/v au dict de destination d_word
      for vk_w_kv in vk_w_dict:
        d_word = {}
        # clé kv
        d_word["word_rec_id"] = vk_w_kv['word_rec_id']
        d_word["word"] = vk_w_kv['word']
        d_word["word_translation"] = vk_w_kv['word_translation']
        for vkcol in vk_word_cols:
          d_word[vkcol] = vk_w_kv[vkcol]
        vk_dict_story_phrase_words.append(d_word)
  return vk_dict_story_phrase_words


In [ ]:
df_work = py_modules.load_df_from_gsheet (wb_intgr, "tbl_phrases")

In [ ]:
df_work

,phrase_rec_id,phrase,phrase_words,phrase_words_rec_id,phrase_html,phrase_html_rec_id,phrase_html_kw,phrase_related_story,phrase_related_story_rec_id,phrase_paragraph,...,phrase_audio_url,phrase_audio_url_fr,phrase_notes_translation,phrase_notes_translation_status,phrase_notes_2_translation,phrase_notes_2_translation_status,phrase_words_tr_rec_id,phrase_words_tr,phrase_related_story_tr_rec_id,phrase_related_story_tr
0,recwaqFTifIlQHW71,Bonjour ! Voici l’ordonnance que vient de me d...,['Ordonnance'],['recl3sIb607a9RrF4'],"Bonjour ! Voici l’<span id=""recl3sIb607a9RrF4""...",Bonjour ! Voici l’<kw>recl3sIb607a9RrF4<kw> qu...,Bonjour ! Voici l’<kw>ordonnance<kw> que vient...,['À la pharmacie'],['rec4nBx7PmDA3Y7Ji'],1,...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,,,,,['recl3sIb607a9RrF4-ams'],['روشتة'],['rec4nBx7PmDA3Y7Ji-ams'],['في الصيدلية']
1,rec2AwFxRzqf9PdyW,"Bonjour monsieur, merci. Pourriez-vous me donn...",['Carte vitale'],['recWrv0IBZdjZZs2L'],"Bonjour monsieur, merci. Pourriez-vous me donn...","Bonjour monsieur, merci. Pourriez-vous me donn...","Bonjour monsieur, merci. Pourriez-vous me donn...",['À la pharmacie'],['rec4nBx7PmDA3Y7Ji'],2,...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,,,,,['recWrv0IBZdjZZs2L-ams'],['بطاقة حيوية'],['rec4nBx7PmDA3Y7Ji-ams'],['في الصيدلية']
2,recZj0qVCbvBFOopO,"Oui, je viens de l’apprendre. Voici ma carte v...",['Carte vitale'],['recWrv0IBZdjZZs2L'],"Oui, je viens de l’apprendre. Voici ma <span i...","Oui, je viens de l’apprendre. Voici ma <kw>rec...","Oui, je viens de l’apprendre. Voici ma <kw>car...",['À la pharmacie'],['rec4nBx7PmDA3Y7Ji'],3,...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,,,,,['recWrv0IBZdjZZs2L-ams'],['بطاقة حيوية'],['rec4nBx7PmDA3Y7Ji-ams'],['في الصيدلية']
3,recd3sgLE5D8rl01O,"Veuillez patienter, je vais chercher vos médi...","['Médicaments', 'Lecteur de glycémie']","['recnLHQQYTu3GDSr8', 'recwvBce8lAbpMjeC']","Veuillez patienter, je vais chercher vos <spa...","Veuillez patienter, je vais chercher vos <kw>...","Veuillez patienter, je vais chercher vos <kw>...",['À la pharmacie'],['rec4nBx7PmDA3Y7Ji'],4,...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,,,,,"['recnLHQQYTu3GDSr8-ams', 'recwvBce8lAbpMjeC-a...","['أدوية', 'مقياس السكر فى الدم']",['rec4nBx7PmDA3Y7Ji-ams'],['في الصيدلية']
4,rec1oH0KOrkPrfjCs,"Je ne parle pas bien français, j'ai peur de ma...",['Ordonnance'],['recl3sIb607a9RrF4'],"Je ne parle pas bien français, j'ai peur de ma...","Je ne parle pas bien français, j'ai peur de ma...","Je ne parle pas bien français, j'ai peur de ma...",['À la pharmacie'],['rec4nBx7PmDA3Y7Ji'],5,...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,,,,,['recl3sIb607a9RrF4-ams'],['روشتة'],['rec4nBx7PmDA3Y7Ji-ams'],['في الصيدلية']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
365,recPiZ0PPxsx3o1a7,"Oui, cela va m’aider car je suis perdu.",['Diabète'],['recmhUC9Wr0zn4i0o'],"Oui, cela va m’aider car je suis perdu.","Oui, cela va m’aider car je suis perdu.","Oui, cela va m’aider car je suis perdu.",['Chez le médecin'],['rec7lQw6DtIJP45dn'],19,...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,,,,,['recmhUC9Wr0zn4i0o-rus'],['Диабет'],['rec7lQw6DtIJP45dn-rus'],['У врача']
366,recXg83ds1lxtoLjK,Et est-ce que vous pouvez envisager de marcher...,['Diabète'],['recmhUC9Wr0zn4i0o'],Et est-ce que vous pouvez envisager de marcher...,Et est-ce que vous pouvez envisager de marcher...,Et est-ce que vous pouvez envisager de marcher...,['Chez le médecin'],['rec7lQw6DtIJP45dn'],20,...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,http://res.cloudinary.com/dhc7ovnwk/video/uplo...,,,,,['recmhUC9Wr0zn4i0o-rus'],['Диабет'],['rec7lQw6DtIJP4

In [ ]:
def df_to_dict__story_phrases (df, vk_cols, filter_language, filter_story):
  print (f'>> df_to_dict__story_phrases : {filter_language} {filter_story}')
  debug = True
  vk_dict_returned = []    # on crée un vecteur (liste) pour recevoir les phrases
  # on filtre sur langue ET story
  df_filtered = df.loc[(df['phrase_language']==filter_language) & (df['phrase_related_story']==filter_story)]
  # si le vecteur de mots est vide, liste vide plutôt que NaN
  #df_filtered['phrase_words_rec_id'] = df_filtered['phrase_words_rec_id'].map(lambda x:ast.literal_eval(x) if x else [])
  # on créé un vecteur de tuples sur cette liste filtrée
  vk_vals = df_filtered.groupby(["phrase_language","phrase_related_story"])[vk_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  # on itère le vecteur de phrases => transforme le vecteur de tuples en liste de dicts
  for vk in vk_vals:
    if debug:
      print (f'phrase={vk}')
    vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
    # on affecte chaque k/v au dict de destination d_phrase
    for vk_kv in vk_dict:
      d_phrase = {}
      for vkcol in vk_cols:
        d_phrase[vkcol] = vk_kv[vkcol]

        ## on ajoute les mots
      # si pas de mots
      if d_phrase["phrase_words_rec_id"]:
        list_words = ast.literal_eval(d_phrase["phrase_words_rec_id"])
        d_phrase["words"] = add_dict_of_words (df_words, vk_word_cols, filter_language, list_words)
      vk_dict_returned.append(d_phrase)
  return vk_dict_returned

In [ ]:
def df_to_dict__tp_phrases (df, vk_cols, filter_language, filter_story):
  print (f'>> add_dict_of_tp_phrases : {filter_language} {filter_story}')
  debug = False
  vk_dict_returned = []    # on crée un vecteur (liste) pour recevoir les phrases
  # on filtre sur langue ET story
  """
  df_filtered = df.loc[(df['tp_phrase_language']==filter_language) & (df['tp_phrase_rel_tp']==filter_story)]
  # si le vecteur de mots est vide, liste vide plutôt que NaN
  df_filtered['tp_phrase_words_rec_id'] = df_filtered['tp_phrase_words_rec_id'].map(lambda x:ast.literal_eval(x) if x else [])
  # on créé un vecteur de tuples sur cette liste filtrée
  vk_vals = df_filtered.groupby(["tp_phrase_language","tp_phrase_rel_tp"])[vk_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  # on itère le vecteur de phrases => transforme le vecteur de tuples en liste de dicts
  for vk in vk_vals:
    # print (f'phrase={vk_phr}')
    vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
    # on affecte chaque k/v au dict de destination d_phrase
    for vk_kv in vk_dict:
      d_phrase = {}
      for vkcol in vk_cols:
        d_phrase[vkcol] = vk_kv[vkcol]

        ## on ajoute les mots
      list_words = d_phrase['tp_phrase_words_rec_id']
      d_phrase["words"] = add_dict_of_words (df_words, vk_word_cols, filter_language, list_words)
      vk_dict_returned.append(d_phrase)
  """
  return vk_dict_returned

In [ ]:
def df_to_dict__quiz (df, vk_cols, filter_language, filter_level):
  print (f'>> df_to_dict__quiz : {filter_language} {filter_level}')
  vk_dict_returned = []
  df_filtered = df.loc[(df['quiz_language']==filter_language) & (df['quiz_rel_story']==filter_level)]
  if len(df_filtered) > 0:
    vk_vals = df_filtered.groupby(["quiz_language","quiz_name"])[vk_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()
    for vk in vk_vals:
      vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
      for vk_quiz_kv in vk_dict:
        d_key = {}
        for vkcol in vk_cols:
          d_key[vkcol] = vk_quiz_kv[vkcol]

      vk_dict_returned.append(d_key)

  return vk_dict_returned


In [ ]:
def df_to_dict__tp (df, vk_cols, filter_language, filter_level):
  print (f'>> df_to_dict__tp : {filter_language} {filter_level}')
  vk_dict_returned = []
  df_filtered = df.loc[(df['tp_language']==filter_language) & (df['tp_rel_story']==filter_level)]
  if len(df_filtered) > 0:
    vk_vals = df_filtered.groupby(["tp_language","tp_name"])[vk_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()
    for vk in vk_vals:
      vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
      d_key = {}

      for vkcol in vk_cols:
        d_key[vkcol] = list(dict.fromkeys(map(
          lambda x:x[vkcol],
          sorted(vk_dict, key=lambda x: x["tp_rec_id"]))))[0]

      d_key['tp_phrases'] = df_to_dict__tp_phrases(
          df_tp_p_ext,
          vk_col_tp_p_ext,
          filter_language,
          d_key['tp_name']
          )

    vk_dict_returned.append(d_key)

  return vk_dict_returned


## Niveau Stories

In [ ]:
def df_to_dict__stories (df, vk_cols, filter_language, filter_level):
  print (f'>> df_to_dict__stories : {filter_language} {filter_level}')
  vk_dict_returned = []
  df_filtered = df.loc[(df['story_language']==filter_language) & (df['story_related_lesson']==filter_level)]
  if len(df_filtered) > 0:
    vk_vals = df_filtered.groupby(["story_rec_id"])[vk_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()
    for vk in vk_vals:
      vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
      d_key = {}

      ## colonnes suppl.
      for vkcol in vk_cols:
        d_key[vkcol] = list(dict.fromkeys(map(
          lambda x:x[vkcol],
          sorted(vk_dict, key=lambda x: x["story_rec_id"]))))[0]

      ## on ajoute les phrases
      d_key['phrases'] = df_to_dict__story_phrases (
          df_phr_ext,
          vk_phrases_cols,
          filter_language,
          d_key["story_name"]
          )
      ## on ajoute le TP
      d_key['tps'] = df_to_dict__tp (
          df_tps_ext,
          vk_tps_cols,
          filter_language,
          d_key["story_name"]
          )
      ## on ajoute le Quiz
      d_key['quiz'] = df_to_dict__quiz (
          df_quiz_ext,
          vk_quiz_cols,
          filter_language,
          d_key["story_name"]
          )

      vk_dict_returned.append(d_key)

  return vk_dict_returned


## Niveau Lesson

In [ ]:
def df_to_dict__lessons (df, vk_cols, filter_language, filter_level):
  print (f'>> df_to_dict__lessons : {filter_language} {filter_level}')
  vk_dict_returned = []
  df_filtered = df.loc[(df['lesson_language']==filter_language) & (df['lesson_related_theme']==filter_level)]
  if len(df_filtered) > 0:
    vk_vals = df_filtered.groupby(["lesson_rec_id"])[vk_cols].apply(
        lambda x: list(map(tuple, x.values.tolist()))).to_list()
    for vk in vk_vals:
      vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
      d_key = {}

      ## colonnes suppl.
      for vkcol in vk_cols:
        d_key[vkcol] = list(dict.fromkeys(map(
          lambda x:x[vkcol],
          sorted(vk_dict, key=lambda x: x["lesson_rec_id"]))))[0]

      # on ajoute les stories
      d_key['stories'] = df_to_dict__stories (
        df_stories,
        vk_stories_cols,
        d_key['lesson_language'],
        d_key["lesson_rec_id"])

    vk_dict_returned.append(d_key)

  return vk_dict_returned


## Niveau Thème

In [ ]:
def df_to_dict__themes (df, vk_cols, filter_language):
  print (f'>> df_to_dict__themes : {filter_language}')
  vk_dict_returned = []
  df_filtered = df.loc[df['theme_language']==filter_language]
  vk_vals = df_filtered.groupby(["theme_rec_id"])[vk_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()
  for vk in vk_vals:
    vk_dict = list(map(lambda x: dict(zip(vk_cols,x)), vk))
    d_key = {}

    ## colonnes suppl.
    for vkcol in vk_cols:
      d_key[vkcol] = list(dict.fromkeys(map(
        lambda x:x[vkcol],
        sorted(vk_dict, key=lambda x: x["theme_rec_id"]))))[0]

    # on ajoute les lessons
    d_key['lessons'] = df_to_dict__lessons (
        df_lessons,
        vk_lessons_cols,
        d_key['theme_language'],
        d_key["theme_rec_id"]
        )

    vk_dict_returned.append(d_key)

  return vk_dict_returned


In [ ]:
vk_langdeck = []
### languages
for vk_l in vk_lang_vals:
  vk_l_dict = list(map(lambda x: dict(zip(vk_language_cols,x)), vk_l))
  d_lang = {}
  d_lang["language"] = list(dict.fromkeys(map(
      lambda x:x["lang_id"],
      sorted(vk_l_dict, key=lambda x: x["lang_id"]))))[0]
  ## colonnes suppl.
  for vkcol in vk_language_cols:
    d_lang[vkcol] = list(dict.fromkeys(map(
      lambda x:x[vkcol],
      sorted(vk_l_dict, key=lambda x: x["lang_id"]))))[0]
  ## theme is the entry point
  d_lang['themes'] = df_to_dict__themes (
      df_themes,
      vk_themes_cols,
      d_lang['language'])
  vk_langdeck.append (d_lang)


>> df_to_dict__themes : ams
>> df_to_dict__lessons : ams recY3WUJq96PZuRPK
>> df_to_dict__stories : ams recyI11Lc7cqVNH12
>> df_to_dict__story_phrases : ams À la pharmacie
phrase=[('ams', 'À la pharmacie', 'rec4nBx7PmDA3Y7Ji', '01', '01', 'recwaqFTifIlQHW71', 'Bonjour ! Voici l’ordonnance que vient de me donner le médecin.', "['Ordonnance']", "['recl3sIb607a9RrF4']", "['روشتة']", "['recl3sIb607a9RrF4-ams']", 'Bonjour ! Voici l’<span id="recl3sIb607a9RrF4">ordonnance</span> que vient de me donner le médecin.', 'Bonjour ! Voici l’<kw>recl3sIb607a9RrF4<kw> que vient de me donner le médecin.', 'Bonjour ! Voici l’<kw>ordonnance<kw> que vient de me donner le médecin.', 'صباح الخير ! هذه هي الوصفة الطبية التي أعطاني إياها الطبيب.', 'assets/audio/ai/ams/recwaqFTifIlQHW71-ams.mp3', 'http://res.cloudinary.com/dhc7ovnwk/video/upload/v1712492028/langdeck/assets/audio/ai/ams/recwaqFTifIlQHW71-ams.mp3', 'http://res.cloudinary.com/dhc7ovnwk/video/upload/v1712492072/langdeck/assets/audio/ai/fre/recwaq

Ensuite, on itère sur les stories filtrées sur une langue

In [ ]:
with open('themes_deck.txt', 'w') as f:
  json.dump(vk_langdeck, f, ensure_ascii=False)

In [ ]:
len(vk_langdeck)

7

In [ ]:
vk_langdeck[4]

In [ ]:
vk_langdeck[4]['themes'][0]['lessons'][0]['stories'][1]

### Requêtes Base Airtable

In [ ]:
saynete = 'Chez le médecin'
df_work = py_modules.load_df_from_gsheet(wb_intgr, "tbl_phrases") # load source sheet
df_work = df_work.loc[df_work['phrase_language']=="fre"]

df_work['phrase_related_story'] = df_work['phrase_related_story'].map(lambda x:ast.literal_eval(x) if x else []).map(lambda x:''.join (x))
df_work = df_work.loc[df_work['phrase_related_story']==saynete]

df_work['phrase_words'] = df_work['phrase_words'].map(lambda x:ast.literal_eval(x) if x else []).map(lambda x:''.join (x))
#df_work['phrase_words_rec_id'] = df_work['phrase_words_rec_id'].map(lambda x:ast.literal_eval(x) if x else []).map(lambda x:''.join (x))

df_work["phrase_position"] = df_work["phrase_position"].str.zfill(2)
df_work = df_work.sort_values(by=['phrase_position'])


In [ ]:
liste_mots_cle = df_work['phrase_words'].to_list()
if 'void' in liste_mots_cle:
  liste_mots_cle.remove('void')
liste_mots_cle = list(dict.fromkeys(liste_mots_cle))

In [ ]:
df_motcle = py_modules.load_df_from_gsheet(wb_intgr, "tbl_words") # load source sheet
df_motcle = df_motcle.loc[df_motcle['word_language']=="fre"]
df_motcle = df_motcle[df_motcle['word'].isin(liste_mots_cle)]

In [ ]:
mask = df_motcle['word'].isin(liste_mots_cle)

In [ ]:
df_motcle = df_motcle [mask]

In [ ]:
df_motcle['traduction_motcle']=''
df_motcle['traduction_def']=''
df_motcle = df_motcle[['word', 'traduction_motcle', 'word_notes', 'traduction_def', ]]
df_motcle.to_csv('saynete_1__motcle.csv', index=False)

In [ ]:
df_work['traduction']=''
df_work = df_work[['phrase_position', 'phrase', 'traduction', ]]
df_work.to_csv('saynete_1__phrases.csv', index=False)

# FIN

In [ ]:
df_work['phrase_related_story'] = df_work['phrase_related_story'].map(lambda x:ast.literal_eval(x) if x else []).map(lambda x:''.join (x))
df_work = df_work.loc[df_work['phrase_related_story']=="Chez la diététicienne"]
df_work = df_work.loc[df_work['phrase_language']=="fre"]
df_work["phrase_position"] = df_work["phrase_position"].str.zfill(2)
df_work = df_work.sort_values(by=['phrase_position'])
df_work['traduction']=''
df_work = df_work[['phrase_position', 'phrase', 'traduction', ]]
df_work.to_csv('saynete_1__phrases.csv', index=False)

In [ ]:
def get_tps (language, story, vk_tuple_tps, vk_tps_cols):
  vk_lang_tp_dict = []
  for vk in vk_tuple_tps:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_tps_cols,x)), vk))
    #print (vk_temp[0]["tp_language"])
    if vk_temp[0]["tp_language"] == language and vk_temp[0]["tp_rel_story"] == story:
      vk_lang_tp_dict = vk_temp
  return vk_lang_tp_dict

In [ ]:
for vk_s in vk_stories_vals:
  vk_s_dict = list(map(lambda x: dict(zip(vk_stories_cols,x)), vk_s))
  d_story = {}
  d_story["story"] = list(dict.fromkeys(map(
    lambda x:x["story_name"],
    sorted(vk_s_dict, key=lambda x: x["story_name"]))))[0]
  print (d_story)

{'story': 'À la pharmacie'}
{'story': 'Chez le médecin'}


## Themes

In [ ]:
df_themes = py_modules.load_df_from_gsheet(wb_intgr, "tbl_themes")
df_themes.sort_values (
    by=["theme_name",
        "theme_language",
        ],
    inplace=True)

vk_themes_cols = [
    "theme_rec_id",
    "theme_name",
    "theme_language",
    "theme_translation",
    "theme_audio",
    "theme_audio_url",
    "theme_audio_url_fr",
    ]

vk_themes_vals = df_themes.groupby(["theme_rec_id"])[vk_themes_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Lessons

In [ ]:
df_lessons = py_modules.load_df_from_gsheet(wb_intgr, "tbl_lessons")
df_lessons.sort_values (
    by=["lesson_number",
        "lesson_language",
        ],
    inplace=True)

vk_lessons_cols = [
    "lesson_rec_id",
    "lesson",
    "lesson_language",
    "lesson_story",
    "lesson_pictorial",
    "lesson_quiz",
    "lesson_number",
    "lesson_translation",
    "lesson_audio",
    "lesson_audio_url",
    "lesson_audio_url_fr",
    "lesson_illustration",
    "lesson_related_theme",
    ]

vk_lessons_vals = df_lessons.groupby(["lesson_rec_id"])[vk_lessons_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Stories

In [ ]:
df_stories = py_modules.load_df_from_gsheet(wb_intgr, "tbl_stories")
df_stories.sort_values (
    by=["story_language",
        "story_level",
        "story_name",
        "story_episode",],
    inplace=True)
vk_stories_cols = [
    "story_rec_id",
    "story_translation_id",
    "story_name",
    "story_translation",
    "story_level",
    "story_episode",
    "story_audio_url",
    "story_audio_url_fr",
    "story_illustration",
    "story_desc",
    "story_desc_translation",
    "story_synopsis",
    "story_synopsis_translation",
    "story_lesson_header",
    "story_related_lesson",
    ]

vk_stories_vals = df_stories.groupby(["story_rec_id"])[vk_stories_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## TPs

In [ ]:
def get_tps (language, story, vk_tuple_tps, vk_tps_cols):
  vk_lang_tp_dict = []
  for vk in vk_tuple_tps:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_tps_cols,x)), vk))
    #print (vk_temp[0]["tp_language"])
    if vk_temp[0]["tp_language"] == language and vk_temp[0]["tp_rel_story"] == story:
      vk_lang_tp_dict = vk_temp
  return vk_lang_tp_dict

df_tps = py_modules.load_df_from_gsheet(wb_intgr, "tbl_tps")
df_tps.sort_values (
    by=["tp_name",
        "tp_language",
        ],
    inplace=True)

vk_tps_cols = [
    "tp_rec_id",
    "tp_name",
    "tp_language",
    "tp_rel_story",
    "tp_rel_story_rec_id",
    "tp_illustration",
    "tp_translation",
    "tp_translation_id",
    "tp_audio",
    "tp_audio_url",
    "tp_audio_url_fr",
    "tp_lesson_header",
    "tp_summary",
    "tp_summary_translation",
    ]

#vk_tps_vals = df_tps.groupby(["tp_rec_id"])[vk_tps_cols].apply(
#      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_tps["tp_rel_story_rec_id"] = df_tps['tp_rel_story_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_tps["tp_rel_story"] = df_tps['tp_rel_story'].map(lambda x: "".join(map(str, ast.literal_eval(x))))

df_tps["tp_rel_story_tr_rec_id"] = df_tps[['tp_rel_story_rec_id', 'tp_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)

# merge phrases + story => vue à plat avec les données étendues
df_tps_ext = pd.merge(df_tps, df_stories[vk_stories_cols],
         left_on="tp_rel_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")

# vector of values grouped by language
vk_tps_vals = df_tps_ext.groupby(["tp_language","tp_name"])[vk_tps_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

### Phrases des TPs

In [ ]:
def get_tp_phrases (language, tp, vk_tuple_tps, vk_tps_cols):
  vk_lang_tp_dict = []
  for vk in vk_tuple_tps:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_tps_cols,x)), vk))
    #print (vk_temp[0]["tp_language"])
    if vk_temp[0]["tp_phrase_language"] == language and vk_temp[0]["tp_rec_id"] == tp:
      vk_lang_tp_dict = vk_temp
  return vk_lang_tp_dict


df_tp_phrases = py_modules.load_df_from_gsheet(wb_intgr, "tbl_tp_phrases")
df_tp_phrases.sort_values (
    by=["tp_phrase_language",
        "tp_phrase_order",
        ],
    inplace=True)

vk_tp_p_cols = [
    "tp_phrase_rec_id",
    "tp_phrase",
    "tp_phrase_html",
    "tp_phrase_html_rec_id",
    "tp_phrase_html_kw",
    "tp_phrase_rel_tp",
    "tp_phrase_rel_tp_rec_id",
    "tp_phrase_order",
    "tp_phrase_language",
    "tp_phrase_translation_id",
    "tp_phrase_translation",
    "tp_phrase_audio",
    "tp_phrase_audio_url",
    "tp_phrase_audio_url_fr",
    "tp_phrase_words_rec_id",
    "tp_phrase_words",
    "tp_phrase_words_tr_rec_id",
    "tp_phrase_words_tr",
    "phrase_related_tp_tr",
    ]

vk_tp_p_vals = df_tp_phrases.groupby(["tp_phrase_rec_id"])[vk_tp_p_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_tp_phrases["tp_phrase_rel_tp_rec_id"] = df_tp_phrases['tp_phrase_rel_tp_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_tp_phrases["tp_phrase_rel_tp"] = df_tp_phrases['tp_phrase_rel_tp'].map(lambda x: "".join(map(str, ast.literal_eval(x))))

# merge phrases + story => vue à plat avec les données étendues
df_tp_p_ext = pd.merge(df_tp_phrases, df_tps,
         left_on="tp_phrase_rel_tp_rec_id",
         right_on="tp_translation_id",
         how="left")

vk_col_tp_p_ext = vk_tp_p_cols + vk_tps_cols + [
  "tp_phrase_words_tr_rec_id",
  "tp_phrase_words_tr",
  "tp_rel_story_tr_rec_id",
  ]

# vector of values grouped by language
vk_tp_p_vals = df_tp_p_ext.groupby(["tp_phrase_language","tp_phrase_rel_tp"])[vk_col_tp_p_ext].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

#### Mots des phrases des TPs

In [ ]:
def get_tp_phrase_words (vk_tp_phrases):
  for tp_phrase in vk_tp_phrases:
    print (tp_phrase["tp_phrase_words_tr_rec_id"])
    vk_words = []
    if len(tp_phrase["tp_phrase_words_tr_rec_id"])>0:
      list_words = ast.literal_eval(tp_phrase["tp_phrase_words_tr_rec_id"])
      for word in list_words:
        #------W O R D S----------
        d_word = {}
        # on récupère la valeur pour chaque attribut de mot
        for field in vk_word_cols:
          d_word[field] = f_words_get_value_by_col(
              df_words,
              "word_translation_id",
              field,
              word)

        d_word["word_tr_rec_id"] = word
        vk_words.append(d_word)
    tp_phrase["words"] = vk_words

  return vk_tp_phrases

"""
vk_tp_phrases = get_tp_phrases(
    d_lang["language"],
    vk_tps[0]["tp_rec_id"],
    vk_tp_p_vals,
    vk_col_tp_p_ext,
    )
get_tp_phrase_words (vk_tp_phrases)
"""

'\nvk_tp_phrases = get_tp_phrases(\n    d_lang["language"],\n    vk_tps[0]["tp_rec_id"],\n    vk_tp_p_vals,\n    vk_col_tp_p_ext,\n    )\nget_tp_phrase_words (vk_tp_phrases)\n'

## Quiz

In [ ]:
def get_quiz (language, tp, vk_tuple_quiz, vk_quiz_cols):
  vk_lang_quiz_dict = []
  for vk in vk_tuple_quiz:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_quiz_cols,x)), vk))
    if vk_temp[0]["quiz_language"] == language and vk_temp[0]["quiz_rel_tp"] == tp:
      vk_lang_quiz_dict = vk_temp
  return vk_lang_quiz_dict

def get_quiz_by_story (language, story, vk_tuple_quiz, vk_quiz_cols):
  vk_lang_quiz_dict = []
  for vk in vk_tuple_quiz:
    # on transforme le tuple en dict
    vk_temp = list(map(lambda x: dict(zip(vk_quiz_cols,x)), vk))
    if vk_temp[0]["quiz_language"] == language and vk_temp[0]["quiz_rel_story"] == story:
      vk_lang_quiz_dict = vk_temp
  return vk_lang_quiz_dict

In [ ]:
df_quiz = py_modules.load_df_from_gsheet(wb_intgr, "tbl_quiz")
df_quiz.sort_values (
    by=["quiz_name",
        "quiz_language",
        ],
    inplace=True)

In [ ]:
vk_quiz_cols = [
    "quiz_rec_id",
    "quiz_name",
    "quiz_language",
    "quiz_rel_tp",
    "quiz_rel_tp_rec_id",
    "quiz_rel_story",
    "quiz_rel_story_rec_id",
    "quiz_illustration",
    "quiz_translation",
    "quiz_translation_id",
    "quiz_audio",
    "quiz_audio_url",
    "quiz_audio_url_fr",
    "quiz_summary_translation",
    "quiz_form",
    "quiz_form_translation",
    "quiz_lesson_header",
    ] + vk_stories_cols


In [ ]:
#vk_quiz_vals = df_quiz.groupby(["quiz_rec_id"])[vk_quiz_cols].apply(
#      lambda x: list(map(tuple, x.values.tolist()))).to_list()

# on passe le string contenant le vecteur en un vrai vecteur puis en string (ouf...)
df_quiz["quiz_rel_tp_rec_id"] = df_quiz['quiz_rel_tp_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_tp"] = df_quiz['quiz_rel_tp'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_story_rec_id"] = df_quiz['quiz_rel_story_rec_id'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_rel_story"] = df_quiz['quiz_rel_story'].map(lambda x: "".join(map(str, ast.literal_eval(x))))
df_quiz["quiz_form"] = df_quiz["quiz_form"].map (lambda x:ast.literal_eval(x))
df_quiz["quiz_form_translation"] = df_quiz["quiz_form_translation"].map (lambda x:ast.literal_eval(x))

df_quiz["quiz_rel_tp_tr_rec_id"] = df_quiz[['quiz_rel_tp_rec_id', 'quiz_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)
df_quiz["quiz_rel_story_tr_rec_id"] = df_quiz[['quiz_rel_story_rec_id', 'quiz_language']].apply(
    lambda x: "{}-{}".format(x[0],x[1]),
    axis=1)

In [ ]:
# merge phrases + story => vue à plat avec les données étendues
df_quiz_ext = pd.merge(df_quiz, df_stories[vk_stories_cols],
         left_on="quiz_rel_story_tr_rec_id",
         right_on="story_translation_id",
         how="left")


In [ ]:
# vector of values grouped by language
vk_quiz_vals = df_quiz_ext.groupby(["quiz_language"])[vk_quiz_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Words and keywords

In [ ]:
df_words = py_modules.load_df_from_gsheet(wb_intgr, "tbl_words")
df_words["order"] = df_words["word_type"].map(lambda x:1 if x=="Lemma" else 2 if x=="Expression" else 3)
df_words.sort_values(by=['word_language','word','order',], inplace=True)
vk_word_cols = [
    "word_language",
    "word",
    "word_rec_id",
    "order",
    "word_type",
    "word_translation_id",
    "word_translation",
    "word_lemmas",
    "word_lemmas_rec_id",
    "word_audio",
    "word_audio_url",
    "word_audio_url_fr",
    "word_lemmas_tr_rec_id",
    "word_lemmas_tr",
    "word_notes",
    "word_nutri_notes",
    "word_notes_translation",
    "word_nutri_notes_translation",
    ] # fields (columns)

vk_word_vals = df_words.groupby(["word_language"])[vk_word_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

## Keywords

In [ ]:
def f_phrases_list_to_dict_by_col(col_name, vkp):
  vkf = list(dict.fromkeys(list(map(
        lambda x: x[col_name],
        filter(lambda d: d['phrase_rec_id'] in vkp,
               sorted(vk_lang_dict, key=lambda x: x["phrase_paragraph"]))))))[0]
  return vkf

def f_words_get_value_by_col (df, col_key, col_name, key):
  cval = df.loc[df[col_key]==key][col_name].values.item()
  return cval


## Languages

In [ ]:
df_languages = py_modules.load_df_from_gsheet (wb_intgr, "tbl_languages")
df_languages["lang_is_tts"] = df_languages["lang_is_tts"].map(lambda x:True if x=="TRUE" else False)
df_languages["lang_is_available"] = df_languages["lang_is_available"].map(lambda x:True if x=="TRUE" else False)
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages")
# merge pour récupérer des infos de référentiels
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","country_ref","flag_icon"]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid",], axis=1, inplace=True)
df_languages.rename(
    columns={"country_ref":"lang_country_ref",
             "flag_icon":"lang_flag_icon"},
    inplace=True)
# merge pour récupérer des infos de référentiels
df_ref_lang = py_modules.load_df_from_gsheet (wb_intgr, "ref_languages_countries")
df_languages = pd.merge (df_languages,
          df_ref_lang[["language_uid","language_countries",]],
          left_on="lang_id",
          right_on="language_uid",
          how="left")
df_languages.drop(columns=["language_uid","lang_countries_vk",], axis=1, inplace=True)
df_languages.rename(
    columns={"language_countries":"lang_countries_vk",},
    inplace=True)
# Liste des colonnes pour le dict
vk_language_cols = [
    "lang_id",
    "lang_alpha3",
    "lang_alpha2_google",
    "lang_alpha2",
    "lang_wals",
    "lang_name_native",
    "lang_name_en",
    "lang_name_fr",
    "lang_is_available",
    "lang_is_tts",
    "lang_countries_vk",
    "lang_country_ref",
    "lang_flag_icon",
]
vk_lang_vals = df_languages.groupby(["lang_id"])[vk_language_cols].apply(
      lambda x: list(map(tuple, x.values.tolist()))).to_list()

In [ ]:
len(vk_stories)

1

In [ ]:
vk_lang = []
# pour chaque phrase : on itère
for vk_lang_tuple in vk_phrases_vals:
  # on transforme le tuple en dict
  vk_lang_dict = list(map(lambda x: dict(zip(vk_phrases_cols,x)), vk_lang_tuple))
  d_lang = {}
  #------L A N G U A G E S----------
  d_lang["language"] = list(dict.fromkeys(map(
      lambda x:x["phrase_language"],
      sorted(vk_lang_dict, key=lambda x: x["phrase_language"]))))[0]
  # get all language cols by key
  for field in vk_language_cols:
    d_lang[field] = df_languages.loc[df_languages["lang_id"]==d_lang["language"]][field].values.item()
  # ce vecteur est dans un format string, il faut le caster en array
  d_lang["lang_countries_vk"] = ast.literal_eval(d_lang["lang_countries_vk"])
  # pour chaque story
  vk_stories = []
  for el_story in list(dict.fromkeys(list(map(
      lambda x: x["phrase_related_story"],
      sorted(vk_lang_dict, key=lambda x: x["phrase_related_story"]))))):
    d_story = {}
    #------S T O R I E S----------
    d_story["story"] = el_story

    #------ T P S----------
    # pour chaque TP
    vk_tps = get_tps(d_lang["language"],
                     el_story,
                     vk_tps_vals,
                     vk_tps_cols,
                     )



    # on ajoute les phrases associées
    vk_tps[0]["tp_phrases"] = get_tp_phrases(
        d_lang["language"],
        vk_tps[0]["tp_rec_id"],
        vk_tp_p_vals,
        vk_col_tp_p_ext,
        )
    # on ajoute les mots récursivement
    vk_tp_phrases = get_tp_phrases(
      d_lang["language"],
      vk_tps[0]["tp_rec_id"],
      vk_tp_p_vals,
      vk_col_tp_p_ext,
      )

    vk_tps = get_tp_phrase_words (vk_tp_phrases)


    d_story["tp_name"] = vk_tps[0]["tp_name"]
    d_story["tp_illustration"] = vk_tps[0]["tp_illustration"]
    d_story["tp_language"] = vk_tps[0]["tp_language"]
    d_story["tp_translation"] = vk_tps[0]["tp_translation"]
    d_story["tp_audio"] = vk_tps[0]["tp_audio"]
    d_story["tp_audio_url"] = vk_tps[0]["tp_audio_url"]
    d_story["tp_audio_url_fr"] = vk_tps[0]["tp_audio_url_fr"]
    d_story["tp_lesson_header"] = vk_tps[0]["tp_lesson_header"]
    d_story["tp_summary"] = vk_tps[0]["tp_summary"]
    d_story["tp_summary_translation"] = vk_tps[0]["tp_summary_translation"]

    d_story["tps"] = vk_tps
    d_story["tp_quiz"] = get_quiz_by_story(d_lang["language"],
              d_story["story"],
              vk_quiz_vals,
              vk_quiz_cols,
              )
    # d_story["tps"]["tp_phrases"] = vk_tps
    # pour chaque phrase
    vk_phrases = []
    for phrase_rec_id in list(dict.fromkeys(list(map(
        lambda x: x["phrase_rec_id"],
        filter(lambda d: d['phrase_related_story'] == el_story,
               sorted(vk_lang_dict, key=lambda x: int(x["phrase_paragraph"]))))))):
      d_phrase = {}
      #------P H R A S E S----------
      # pour chaque colonne, on tranforme en dict
      for field in vk_phrases_cols:
        d_phrase[field] = f_phrases_list_to_dict_by_col(
            field,
            phrase_rec_id)
      d_phrase["phrase_rec_id"] = phrase_rec_id
      # rendons à story ce qui appartient à story...
      for field in vk_stories_cols:
        d_story[field] = d_phrase[field]
        d_phrase.pop(field)

      # pour chaque mot
      vk_words = []
      if len(d_phrase["phrase_words_tr_rec_id"])>0:
        list_words = ast.literal_eval(d_phrase["phrase_words_tr_rec_id"])
        for word in list_words:
          #------W O R D S----------
          d_word = {}
          # on récupère la valeur pour chaque attribut de mot
          for field in vk_word_cols:
            d_word[field] = f_words_get_value_by_col(
                df_words,
                "word_translation_id",
                field,
                word)

          d_word["word_tr_rec_id"] = word
          # lemmes et expressions
          vk_lem_exp = []
          # si lemmes : on itère le vecteur
          if len(d_word["word_lemmas_rec_id"])>0:
            list_lems = ast.literal_eval(d_word["word_lemmas_rec_id"])
            for lem in list_lems:
              #---K E Y W O R D S---
              d_lem = {}
              # on construit la clé de recherche id + langue
              lemkey = lem + "-" + d_word["word_language"]
              # pour chaque colonne (idem words) : on récupère la valeur à partir de la clé
              for field in vk_word_cols:
                d_lem[field] = f_words_get_value_by_col(
                  df_words,
                  "word_translation_id",
                  field,
                  lemkey)
              #----------------------
              d_lem["keyword"] = lem
              vk_lem_exp.append(d_lem)
              #----------------------

          #----------------------
          d_word["keywords"] = vk_lem_exp
          vk_words.append(d_word)
          #----------------------

      #----------------------
      d_phrase["words"] = vk_words
      vk_phrases.append(d_phrase)
      #----------------------

    #----------------------
    d_story["phrases"] = vk_phrases
    #----------------------
    vk_stories.append(d_story)


  #----------------------
  d_lang["stories"] = vk_stories
  #----------------------

  vk_lang.append(d_lang)


['recRUrPp0aC5oNvVc-dut']



['recRUrPp0aC5oNvVc-eng']



['recRUrPp0aC5oNvVc-fre']



['recRUrPp0aC5oNvVc-ger']



['recRUrPp0aC5oNvVc-rom']





In [ ]:
len(vk_stories)

1

In [ ]:
vk_lang[0]

In [ ]:
vk_tp_phrases

[]

In [ ]:
vk_lang[3]['stories']

## Sauvegarde au format JSON

In [ ]:
with open('saynetes.txt', 'w') as f:
  json.dump(vk_lang, f, ensure_ascii=False)

# Section InDesign
[retour](#scrollTo=Y33GmC59GXP_)

In [ ]:
b_indd_phrases = True
if b_indd_phrases:
  df_tbl_phrases = py_modules.load_df_from_gsheet (wb_intgr, "tbl_phrases")
  df_tbl_phrases['phrase_related_story_rec_id'] = df_tbl_phrases['phrase_related_story_rec_id'].map(lambda x:ast.literal_eval(x) if x else [])
  df_tbl_phrases['phrase_related_story_rec_id'] = df_tbl_phrases['phrase_related_story_rec_id'].map(lambda x:''.join (x))
  # phrase_related_story_rec_id rec4nBx7PmDA3Y7Ji
  df_fr = df_tbl_phrases.loc[(df_tbl_phrases['phrase_language']=='fre') & (df_tbl_phrases['phrase_related_story_rec_id']=='rec7lQw6DtIJP45dn')][['phrase_translation','phrase_paragraph']]
  df_tr = df_tbl_phrases.loc[(df_tbl_phrases['phrase_language']=='rom') & (df_tbl_phrases['phrase_related_story_rec_id']=='rec7lQw6DtIJP45dn')][['phrase_translation','phrase_paragraph']]
  vk_fr =  df_fr.to_dict(orient='records')
  vk_tr =  df_tr.to_dict(orient='records')

  vk = []
  for index, item in enumerate(vk_fr, start=0):   # Python indexes start at zero
      print(f'{index}, {item["phrase_translation"]}, {vk_tr[index]["phrase_translation"]}')
      dk = {}
      dk["order"]=item["phrase_paragraph"]
      dk["phrase"]=item["phrase_translation"]
      vk.append(dk)
      dk = {}
      dk["order"]=""
      dk["phrase"]=vk_tr[index]["phrase_translation"]
      vk.append(dk)
  df = pd.DataFrame(vk)
  py_modules.save_df_to_gsheet (wb_intgr, "tbl_phr_id", df)

0, Bonjour Docteur, je viens vous voir parce qu’on m’a trouvé du diabète aux Urgences., Bună ziua, doctore, am venit să vă văd pentru că am fost descoperită că am diabet la Urgențe.
1, Cela me fait peur car ma mère a été amputée à cause de çà., Acest lucru mă sperie pentru că mama mea a avut o amputație din cauza asta.
2, Bonjour Monsieur, prenez le temps de vous asseoir. Nous allons faire le point ensemble., Bună ziua domnule, fă-ți timp să te așezi. Vom face bilanțul împreună.
3, J'ai ici le compte-rendu des Urgences et il confirme que vous avez du diabète., Am aici raportul de urgență și confirmă că aveți diabet.
4, Votre taux de sucre dans le sang (glycémie) est élevé : 2,8 grammes par litre., Nivelul de zahăr din sânge (glicemia) este ridicat: 2,8 grame pe litru.
5, Comment vous sentez-vous aujourd'hui ?, Cum te simti azi ?
6, Je suis fatigué et je dors mal car je me lève souvent pour faire pipi., Sunt obosită și nu dorm bine pentru că mă trezesc des să fac pipi.
7, Ces signes son